In [3]:
import pickle, json, time, datetime, copy, pymongo, re, spacy, scispacy, requests, ast, \
en_ner_bc5cdr_md, en_ner_bionlp13cg_md, en_ner_jnlpba_md, string, os, random
from pymongo import MongoClient
import pandas as pd
from collections import Counter
from tqdm.notebook import tqdm
tqdm.pandas()
from math import ceil
from itertools import combinations, permutations
from spacy.tokenizer import Tokenizer
from multiprocessing import Process, Manager, Pool
from lxml import etree
from nltk.tokenize import sent_tokenize

In [4]:
now=datetime.datetime.now()

In [6]:
now_time=now.strftime('%Y_%m_%d')

In [11]:
def cal_time(time):
    time = time.split('/')
    year = int(time[0])
    month = int(time[1]) - 3
    day = int(time[2])
    if month < 1:
        month = 12
        year -= 1
    time = datetime.datetime(year=year, month=month, day=1).strftime('%Y/%m/%d')
    return time

In [16]:
client = MongoClient(username="hald",password="mclab236")

In [17]:
db = client.hald

In [18]:
raw_pick=pickle.load(open('results/step1/raw_pick.pkl','rb'))

In [19]:
raw_pick_pubtator=pickle.load(open('results/step1/raw_pick_pubtator.pkl','rb'))

In [20]:
pubtator_raw=pickle.load(open('results/step1/pubtator_raw.pkl','rb'))

In [21]:
pubtator_raw_dict2AB={}
for i in pubtator_raw:
    pubtator_raw_dict2AB[i.get('id')]=i.get('AB')

# Gene_raw

In [22]:
collections_Gene_raw=db.Gene_raw

In [23]:
rawpickwithAB={}
for i in raw_pick_pubtator:
    rawpickwithAB[i.get('PMID')]=i

In [24]:
pubtator_gene=[]
for i in pubtator_raw:
    if i.get('type')=='Gene':
        pubtator_gene.append(i)

In [25]:
len(pubtator_gene)

2152

In [27]:
Gene2ncbi=pd.read_csv('../ncbi/Gene_ncbi.txt',sep='\t',encoding='utf-8',converters = {'NCBI Gene ID(supplied by NCBI)':str})

In [28]:
ncbi2Gene_dict=dict(zip(Gene2ncbi['NCBI Gene ID(supplied by NCBI)'],Gene2ncbi['Approved symbol'] ))

In [29]:
error=[]
pubtator_genes=[]
for i in pubtator_gene:
    if ncbi2Gene_dict.get(i.get('identifier')):
        i['entity']=ncbi2Gene_dict.get(i.get('identifier'))
        pubtator_genes.append(i)
    else:
        error.append(i)

In [30]:
len(pubtator_genes)

1796

In [31]:
Genelist=pickle.load(open('../dict/Genedict_list.pkl','rb'))

In [32]:
Gene_list={}
for i in Genelist:
    Gene_list[i.lower()]=i.lower()

In [33]:
filtered_pubtator_genes=[]
drop_genes=[]
for i in pubtator_genes:
    if Gene_list.get(i.get('text').lower()):
        filtered_pubtator_genes.append(i)
    else:
        drop_genes.append(i)

In [34]:
len(drop_genes)

348

In [35]:
len(filtered_pubtator_genes)

1448

In [36]:
filtered_pubtator_genes[10]

{'id': '37059468',
 'AB': 'Among the glutamic acid decarboxylase (GAD)-antibody-spectrum disorders, the most common phenotypic subset is the stiff-person syndrome (SPS), caused by impaired GABAergic inhibitory neurotransmission and autoimmunity characterized by very high titers of GAD antibodies and increased GAD-IgG intrathecal synthesis. If not properly treated or untreated because of delayed diagnosis, SPS progresses leading to disability; it is therefore fundamental to apply the best therapeutic schemes from the outset. This article is focused on the rationale of specific therapeutic strategies based on the SPS pathophysiology targeting both the impaired reciprocal GABAergic inhibition to symptomatically improve the main clinical manifestations of stiffness in the truncal and proximal limb muscles, gait dysfunction, and episodic painful muscle spasms and the autoimmunity to enhance improvement and slow down disease progression. A practical, step-by-step therapeutic approach is prov

In [37]:
for i in filtered_pubtator_genes:
    i['ner']='HALD_Gene'
    i['PMID']=i.get('id')
    i['word']=i.get('text')

In [39]:
ner_bc5c = en_ner_bc5cdr_md.load()
ner_bionl = en_ner_bionlp13cg_md.load()
ner_jnlp = en_ner_jnlpba_md.load()

In [40]:
def get_entities(ti, model, keep=[]):
    doc = model(ti)
    if len(keep) > 0:
        return [(x.text.replace('\n', ''), x.label_) for x in doc.ents if str(x.label_) in keep]
    return [(x.text.replace('\n', ''), x.label_) for x in doc.ents ]

In [41]:
raw_pick_pubtator_pmid=[]
for i in raw_pick_pubtator:
    if i.get('AB'):
        raw_pick_pubtator_pmid.append(i.get('PMID'))
    else:
        pass

In [42]:
filtered_pubtator_genes[0]

{'id': '37058572',
 'AB': 'Sirtuin 6 (SIRT6) is a multifaceted protein deacetylase/deacylase and a major target for small-molecule modulators of longevity and cancer. In the context of chromatin, SIRT6 removes acetyl groups from histone H3 in nucleosomes, but the molecular basis for its nucleosomal substrate preference is unknown. Our cryo-electron microscopy structure of human SIRT6 in complex with the nucleosome shows that the catalytic domain of SIRT6 pries DNA from the nucleosomal entry-exit site and exposes the histone H3 N-terminal helix, while the SIRT6 zinc-binding domain binds to the histone acidic patch using an arginine anchor. In addition, SIRT6 forms an inhibitory interaction with the C-terminal tail of histone H2A. The structure provides insights into how SIRT6 can deacetylate both H3 K9 and H3 K56.',
 'type': 'Gene',
 'identifier': '51548',
 'text': 'Sirtuin 6',
 'sentence': 'Sirtuin 6 (SIRT6) is a multifaceted protein deacetylase/deacylase and a major target for small-m

prepare_for_scispacy={}
for i in filtered_pubtator_genes:
    if not filtered_pubtator_genes.get(i.get('PMID')):
        filtered_pubtator_genes[i.get('PMID')]=i

In [43]:
def work(i,finallist_tags,lock):
    finallist_tags.append({'PMID':i.get('PMID'), 'AB':i.get('AB'), 'sentence':i.get('sentence'), 'sentenceIndex': i.get('sentenceIndex'),\
                          'sentenceSize':i.get('sentenceSize'),'word': i.get('word'),'identifier':i.get('identifier'),'type':i.get('type'),\
                           'entity':i.get('entity'),'jnlpba_tags':get_entities(i.get('sentence'), model=ner_jnlp, keep=['DNA','RNA','PROTEIN']),\
                          'bc5cdr_tags':get_entities(i.get('sentence'), model=ner_bc5c),\
                          'bionlp13cg_tags':get_entities(i.get('sentence'), model=ner_bionl, \
                          keep=['GENE_OR_GENE_PRODUCT'])})
if __name__=='__main__':
    start=datetime.datetime.now()
    print('Parent process %s.' % os.getpid())
    p=Pool(60)
    manager=Manager()
    finallist_tags=manager.list()
    lock=manager.Lock()
    for i in filtered_pubtator_genes:
        p.apply_async(work, args=(i,finallist_tags,lock))
    print('Waiting for all subprocesses done...')
    p.close()
    p.join()
    print('All subprocesses done.')
    end=datetime.datetime.now()
    print("The running time is "+str((end-start).seconds)+"s")  

Parent process 74860.
Waiting for all subprocesses done...
All subprocesses done.
The running time is 16s


In [44]:
Gene_tags=list(finallist_tags)

In [45]:
len(Gene_tags)

1448

In [47]:
scispicy_Gene=copy.deepcopy(Gene_tags)

In [48]:
for i in scispicy_Gene:
    i['ScispaCy']=[]
    if i.get('jnlpba_tags'):
        for j in i.get('jnlpba_tags'):
            if j[1]=='DNA' or j[1]=='PROTEIN':
                i['ScispaCy']+=[j[0]]
    if i.get('bionlp13cg_tags'):
        for k in i.get('bionlp13cg_tags'):
            i['ScispaCy']+=[k[0]]
    i['ScispaCy']=list(set(i.get('ScispaCy')))

In [49]:
scicpacy_Genes=[]
not_scicpacy_Genes=[]
for i in scispicy_Gene:
    if i.get('ScispaCy'):
        scicpacy_Genes.append(i)
    else:
        not_scicpacy_Genes.append(i)

In [50]:
len(scicpacy_Genes)

1338

In [51]:
len(not_scicpacy_Genes)

110

In [52]:
not_scicpacy=[]
for i in not_scicpacy_Genes:
    not_scicpacy.append({'PMID':i.get('PMID'),'word':i.get('word'),'sentence':i.get('sentence')})

In [53]:
not_scicpacy[-10:-1]

[{'PMID': '34411297',
  'word': 'hub',
  'sentence': 'We analyze key network properties including connectivity, module/hub, and temporal variation.'},
 {'PMID': '34266325',
  'word': 'MoCA',
  'sentence': 'Meanwhile, MoCA was also used to reevaluate the elderly inpatients with normal MMSE scores at different education levels.'},
 {'PMID': '34266325',
  'word': 'MoCA',
  'sentence': 'However, in the cognitive assessment of elderly patients, the ability to identify impairment with the MoCA is obviously superior to that with the MMSE.'},
 {'PMID': '34411297',
  'word': 'hub',
  'sentence': 'For example, high connectivity and hub conditions, such as disorders of lipid metabolism and essential hypertension, are identified.'},
 {'PMID': '34266325',
  'word': 'MoCA',
  'sentence': 'CONCLUSIONS: In general, the results of both the MMSE and MoCA were influenced by education level, and this effect was more obvious for the MoCA.'},
 {'PMID': '34266325',
  'word': 'MoCA',
  'sentence': 'CONCLUSION

In [54]:
scicpacy_Genes_final=[]
drop_scispacy=[]
for i in scicpacy_Genes:
    pattern=i.get('word').translate(str.maketrans({"-":  r"\-","\\": r"\\", "^":  r"\^","$":  r"\$","*":  r"\*",".":  r"\.","(":  r"\(",")":  r"\)","+":  r"\+","[":  r"\[","]":  r"\]",
                              "{":  r"\{","}":  r"\}","|":  r"\|","?":  r"\?"}))
    for j in i.get('ScispaCy'):
        if i.get('word')==j or re.search(pattern, j):
            scicpacy_Genes_final.append(i)
            break
    else:
        drop_scispacy.append(i)

In [55]:
len(scicpacy_Genes_final)

1228

In [56]:
for i in scicpacy_Genes_final:
    if re.search('-',i.get('word')):
        print(i)
        break

{'PMID': '37047803', 'AB': 'Herpesviridae reactivation such as cytomegalovirus (CMV) has been described in severe COVID-19 (COronaVIrusDisease-2019). This study aimed to understand if CMV reactivation in older COVID-19 patients is associated with increased inflammation and in-hospital mortality. In an observational single-center cohort study, 156 geriatric COVID-19 patients were screened for CMV reactivation by RT-PCR. Participants underwent a comprehensive clinical investigation that included medical history, functional evaluation, laboratory tests and cytokine assays (TNF-alpha, IFN-alpha, IL-6, IL-10) at hospital admission. In 19 (12.2%) of 156 COVID-19 patients, CMV reactivation was detected. Multivariate Cox regression models showed that in-hospital mortality significantly increased among CMV positive patients younger than 87 years (HR: 9.94, 95% CI: 1.66-59.50). Other factors associated with in-hospital mortality were C-reactive protein (HR: 1.17, 95% CI: 1.05-1.30), neutrophil c

In [57]:
scicpacy_Genes_final[0]

{'PMID': '37058572',
 'AB': 'Sirtuin 6 (SIRT6) is a multifaceted protein deacetylase/deacylase and a major target for small-molecule modulators of longevity and cancer. In the context of chromatin, SIRT6 removes acetyl groups from histone H3 in nucleosomes, but the molecular basis for its nucleosomal substrate preference is unknown. Our cryo-electron microscopy structure of human SIRT6 in complex with the nucleosome shows that the catalytic domain of SIRT6 pries DNA from the nucleosomal entry-exit site and exposes the histone H3 N-terminal helix, while the SIRT6 zinc-binding domain binds to the histone acidic patch using an arginine anchor. In addition, SIRT6 forms an inhibitory interaction with the C-terminal tail of histone H2A. The structure provides insights into how SIRT6 can deacetylate both H3 K9 and H3 K56.',
 'sentence': 'In addition, SIRT6 forms an inhibitory interaction with the C-terminal tail of histone H2A.',
 'sentenceIndex': 3,
 'sentenceSize': 5,
 'word': 'SIRT6',
 'id

In [58]:
len(drop_scispacy)

110

In [59]:
scicpacy_pmids=[]
for i in scicpacy_Genes_final:
    if i.get('PMID') not in scicpacy_pmids:
        scicpacy_pmids.append(i.get('PMID'))

In [60]:
len(scicpacy_pmids)

235

In [61]:
def query_raw_id(ids, url="https://bern.korea.ac.kr/pubmed/"):
    return requests.get(url + ','.join(ids), verify=False).json()

In [62]:
os.makedirs('../bern/'+now_time)

In [63]:
f=open('../bern/'+now_time+'/download.txt','a')
bern_download_id=1
for j in range(0,len(scicpacy_pmids),5):
    f.write('wget https://bern.korea.ac.kr/pubmed/'+','.join(scicpacy_pmids[j:j+5])+' --no-check-certificate -O '+'../bern/'+now_time+'/bern'+str(bern_download_id)+'.json \n')
    bern_download_id+=1
f.close()

In [64]:
f=open('../bern/'+now_time+'/run.sh','w')
f.write("while read rows; do $rows  ; done < ../bern/"+now_time+'/download.txt')
f.close()

In [65]:
print("nohup bash "+'../bern/'+now_time+'/run.sh > '+'../bern/'+now.strftime('%Y_%m_%d')+'/bern.log &')

nohup bash ../bern/2023_05_09/run.sh > ../bern/2023_05_09/bern.log &


## RUN BERN ON LINUX!!!!!!
### Code is showed above, run it in the current directory.
## Check the file size!!!!!!

In [67]:
all_json=[]
for i in os.listdir('../bern/'+now_time+'/'):
    if i.split('.')[-1] == 'json':
        all_json+=json.load(open('../bern/'+now_time+'/'+i))

In [68]:
allbern=copy.deepcopy(all_json)

In [69]:
len(allbern)

235

In [70]:
bern_json={}
for i in allbern:
    bern_value=[]
    if i.get('denotations'):
        for j in i.get('denotations'):
            if j.get('obj')=='gene' or j.get('obj')=='protein':
                bern_value.append(i.get('text')[j.get('span').get('begin'):j.get('span').get('end')])
    bern_json[i.get('sourceid')]=bern_value    

In [71]:
len(bern_json)

235

In [72]:
count=0
filtered_bern_json={}
for i,j in bern_json.items():
    if j:
        count+=1
        filtered_bern_json[i]=j
print(count)

226


In [73]:
list(filtered_bern_json.items())[0]

('36794912', ['16S', 'SNF-1', 'SNF-1', 'SNF-3', 'SNF-2', 'CD4+'])

In [74]:
SciSpacy_raw=copy.deepcopy(scicpacy_Genes_final)

In [75]:
len(SciSpacy_raw)

1228

In [76]:
SciSpacy_raw_dict={}
for i in SciSpacy_raw:
    SciSpacy_raw_dict[i.get('PMID')]=SciSpacy_raw_dict.get(i.get('PMID'),[])+[i]

In [77]:
len(SciSpacy_raw_dict)

235

In [78]:
cs=0
intersec={}
dropout=[]
for i,j in filtered_bern_json.items():
    if SciSpacy_raw_dict.get(i):
        midd=[]
        for k in SciSpacy_raw_dict.get(i):
            if k.get('word') in j:
                k['bern']=j
                midd.append(k)
            else:
                for m in j:
                    pattern=k.get('word').translate(str.maketrans({"-":  r"\-","\\": r"\\", "^":  r"\^","$":  r"\$","*":  r"\*",".":  r"\.","(":  r"\(",")":  r"\)","+":  r"\+","[":  r"\[","]":  r"\]",
                      "{":  r"\{","}":  r"\}","|":  r"\|","?":  r"\?"}))
                    if re.search(pattern, m):
                        k['bern']=j
                        midd.append(k)
                        break
                else:
                    dropout.append(k)
        intersec[i]=midd
        

In [79]:
len(intersec)

226

In [80]:
len(dropout)

20

In [81]:
final_Gene_raw=[]
for i in intersec.values():
    final_Gene_raw+=i

In [82]:
len(final_Gene_raw)

1179

In [83]:
Genes_inte=copy.deepcopy(final_Gene_raw)

In [84]:
Gene2blank={}
for i in Genes_inte:
    Gene2blank[i.get('entity')]=''

In [85]:
len(Gene2blank)

288

In [86]:
Gene2ncbi=pd.read_csv('../ncbi/Gene_ncbi.txt',sep='\t',encoding='utf-8',converters = {'NCBI Gene ID(supplied by NCBI)':str})

In [87]:
Gene2ncbi_dict=dict(zip(Gene2ncbi['Approved symbol'], Gene2ncbi['NCBI Gene ID(supplied by NCBI)']))

In [88]:
Gene_converter=pickle.load(open('../ncbi/Gene_converter.pkl','rb'))

In [89]:
len(Gene_converter)

4953

In [90]:
Genes_inte_copy=copy.deepcopy(Genes_inte)

In [91]:
rawpickwithAB={}
for raw in raw_pick_pubtator:
    if raw.get('AB'):
        rawpickwithAB[raw.get('PMID')]=raw

In [92]:
Gene_deduplicated=set()
Gene_deduplicated_count=0
Gene_list_raw=[]
for i in Genes_inte_copy:
    k=i.get('PMID')
    i['DP']=rawpickwithAB.get(k).get('DP')
    i['AB']=rawpickwithAB.get(k).get('AB')
    i['JT']=rawpickwithAB.get(k).get('JT')
    i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+k
    i['TI']=rawpickwithAB.get(k).get('TI')
    i['target']=i.get('word')
    i['type']='Gene'
    tuples=(i.get("PMID"),i.get("entity"),i.get("word"),i.get("sentence"))
    if tuples not in Gene_deduplicated:
        Gene_deduplicated.add(tuples)
        Gene_list_raw.append(i)
    else:
        Gene_deduplicated_count+=1
        continue

In [93]:
len(Gene_list_raw)

1056

In [94]:
Gene2blank={}
for i in Gene_list_raw:
    Gene2blank[i.get('entity')]=''

In [95]:
len(Gene2blank)

288

In [96]:
def get_Gene_info(i,final_Genes_inte):
    ncbiid=Gene2ncbi_dict.get(i)
    ncbiurl='https://www.ncbi.nlm.nih.gov/gene/' + ncbiid
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"
    }
    url = 'https://www.ncbi.nlm.nih.gov/gene/'+ncbiid
    res=requests.get(url=url,headers=headers)
    html = etree.HTML(res.content)
    ncbisummary={
    'Official_Symbol' : html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Official")]/following::dd[1]/text()')[0],
    'Official_Full_Name' : html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Official")]/following::dd[1]/text()')[1],
    'HGNC_ID' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Primary source")]/following::dd[1]/a/text()')),
    'HGNC_URL' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Primary source")]/following::dd[1]/a/@href')),
    'Ensembl_ID' :  ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[1]/text()')),
    'Ensembl_URL' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[1]/@href')),
    'MIM_ID' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[2]/text()')),
    'MIM_URL' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[2]/@href')),
    'AllianceGenome_ID' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[3]/text()')),
    'AllianceGenome_URL' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[3]/@href')),
    'NCBI_ID' : ncbiid ,
    'NCBI_URL' : ncbiurl
    }
    try:
        also=''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Also known as")]/following::dd[1]/text()')).replace('HGNC:HGNC:','')
    except:
        also=''
    finally:
        ncbisummary.update({'Also_known_as' : also})
    try:
        also2=''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Summary")]/following::dd[1]/text()'))
    except:
        also2=''
    finally:
        ncbisummary.update({'Summary' : also2})
    final_Genes_inte[i]=ncbisummary
    time.sleep(0.5)

In [97]:
judge=False
p = Pool(10)
manager=Manager()
final_Genes_inte = manager.dict()
for i in Gene2blank.keys():
    if not Gene_converter.get(i):
        p.apply_async(get_Gene_info, args=(i,final_Genes_inte))
        judge=True
p.close()
p.join()
if judge:
    Gene_converter.update(dict(final_Genes_inte))
    pickle.dump(Gene_converter, open('../ncbi/Gene_converter.pkl','wb'))

In [98]:
judge

True

In [99]:
len(Gene_converter)

4988

In [100]:
for i in range(len(Gene_list_raw)):
    Gene_list_raw[i]=dict(Gene_list_raw[i],**Gene_converter.get(Gene_list_raw[i].get('entity')))

In [101]:
dropout=[]
months=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
for i in Gene_list_raw:
    j=i.get('DP')
    if re.search('[S,s]upplement',j,flags=re.M|re.I):
        j=re.sub('[S,s]upplement.*','',j,flags=re.M|re.I)
    j=re.sub('\s?-\s?','-',j)
    j=re.sub('\.','',j)
    j=re.sub('Aout','Aug',j)
    if len(j.split(' '))==3:
        try:
            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
        except:
            if j.split()[1] in months:
                if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                    j=j.split()[0]+' '+j.split()[1]+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                        print('***************************************ERROR**********************************************')
                else:
                    j=j.split()[0]+' '+j.split()[1]+' '+ '1'
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        print(j)
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
            elif j.split()[1] not in months:
                if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                        middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                    else:
                        middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if middle in months:
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+middle+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+middle+' '+ '1'
                    elif re.match(r'Sum(mer)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Jun'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Jun'+' '+ '1'
                    elif re.match(r'Spr(ing)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Mar'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Mar'+' '+ '1'
                    elif re.match(r'Win(ter)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Dec'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Dec'+' '+ '1'    
                    elif re.match(r'Aut(umn)?',middle) or \
                    re.match(r'Fall',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Sep'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Sep'+' '+ '1'      
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        print(j)
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                else:
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    elif len(j.split())==2:
        if re.match(r'Sum(mer)?',j.split()[1]):
            date=int(j[:4]+'0601')
        elif re.match(r'Spr(ing)?',j.split()[1]):
            date=int(j[:4]+'0301')
        elif re.match(r'Win(ter)?',j.split()[1]):
            date=int(j[:4]+'1201')
        elif re.match(r'Aut(umn)?',j.split()[1]) or re.match(r'Fall',j.split()[1]):
            date=int(j[:4]+'0901')
        elif re.match(r'Sum(mer)?',j.split()[0]):
            date=int(j[-4:]+'0601')
        elif re.match(r'Spr(ing)?',j.split()[0]):
            date=int(j[-4:]+'0301')
        elif re.match(r'Win(ter)?',j.split()[0]):
            date=int(j[-4:]+'1201')
        elif re.match(r'Aut(umn)?',j.split()[0]) or re.match(r'Fall',j.split()[0]):
            date=int(j[-4:]+'0901')
        else:
            try:
                date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b'), '%Y%m'+'01'))
            except:
                if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                        middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                    else:
                        middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if middle in months:
                        j=j.split()[0]+' '+middle+' '+ '1'
                    elif re.match(r'Sum(mer)?',middle):
                        j=j.split()[0]+' '+'Jun'+' '+ '1'
                    elif re.match(r'Spr(ing)?',middle):
                        j=j.split()[0]+' '+'Mar'+' '+ '1'
                    elif re.match(r'Win(ter)?',middle):
                        j=j.split()[0]+' '+'Dec'+' '+ '1'    
                    elif re.match(r'Aut(umn)?',middle) or \
                    re.match(r'Fall',middle):
                        j=j.split()[0]+' '+'Sep'+' '+ '1'      
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                elif re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    midd=re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if len(midd)!=2:
                        midd='0'+midd
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+midd+'01'))
                else:
                    print(j)
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    elif len(j.split())==1:
        j=j[:4]
        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    else:
        j=j.split()[0]+' '+j.split()[1]+' '+re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
    i['date']=date
    i['year']=int(str(i.get('date'))[:4])

In [102]:
for i in Gene_list_raw:
    i['addtime']=now_time

In [103]:
Gene_raw=copy.deepcopy(Gene_list_raw)

In [104]:
len(Gene_raw)

1056

In [105]:
if len(Gene_raw)!=0:
    collections_Gene_raw.insert_many(Gene_raw)

# Gene_all

In [106]:
Gene_raw_list=list(collections_Gene_raw.find())

In [107]:
for i in Gene_raw_list:
    i['type']='Gene'
    i['JT']=[i.get('JT')]
    i["PMID"]=[i.get("PMID")]
    i["sentence"]=[[i.get("sentence")]]
    pmid=i.get('PMID')[0]
    i['DP']=[i.get("DP")]
    i['AB']=[i.get("AB")]
    i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+pmid
    i['TI']=[i.get("TI")]
    i['target']=[[i.get('word')]]
    i["date"]=[i.get("date")]
    i["year"]=[i.get("year")]

In [108]:
Gene_mid={}
for i in Gene_raw_list:
    if not Gene_mid.get(i.get("entity")):
        Gene_mid[i.get("entity")]=i
    else:
        if i.get('PMID')[0] in Gene_mid.get(i.get('entity')).get('PMID'):

            ind=Gene_mid.get(i.get('entity')).get('PMID').index(i.get('PMID')[0])
            if i.get("sentence")[0][0] in Gene_mid.get(i.get('entity')).get('sentence')[ind]:
                continue
            else:
                Gene_mid[i.get('entity')]['sentence'][ind] = Gene_mid.get(i.get('entity')).get('sentence')[ind] + i.get("sentence")[0]
                Gene_mid[i.get('entity')]['target'][ind] = Gene_mid.get(i.get('entity')).get('target')[ind] + i.get("target")[0]
        else:
            Gene_mid[i.get("entity")]["PMID"]=Gene_mid.get(i.get("entity")).get("PMID")+i.get("PMID")
            Gene_mid[i.get("entity")]["sentence"]=Gene_mid.get(i.get("entity")).get("sentence")+i.get("sentence")
            Gene_mid[i.get("entity")]["AB"]=Gene_mid.get(i.get("entity")).get("AB")+i.get("AB")
            Gene_mid[i.get("entity")]["DP"]=Gene_mid.get(i.get("entity")).get("DP")+i.get("DP")
            Gene_mid[i.get("entity")]["TI"]=Gene_mid.get(i.get("entity")).get("TI")+i.get("TI")
            Gene_mid[i.get("entity")]["target"]=Gene_mid.get(i.get("entity")).get("target")+i.get("target")
            Gene_mid[i.get("entity")]["date"]=Gene_mid.get(i.get("entity")).get("date")+i.get("date")
            Gene_mid[i.get("entity")]["year"]=Gene_mid.get(i.get("entity")).get("year")+i.get("year")
            Gene_mid[i.get("entity")]["JT"]=Gene_mid.get(i.get("entity")).get("JT")+i.get("JT")

In [109]:
Gene_output_json_final=list(Gene_mid.values())

In [110]:
len(Gene_output_json_final)

4988

In [111]:
for i in Gene_output_json_final:
    i['allyears']=[]
    i['datestring']=[]
    for j in i.get('date'):
        if int(str(j)[:4]) not in i.get('allyears'):
            i['allyears']= i.get('allyears')+[int(str(j)[:4])]
    for j in i.get('date'):
        if str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:] not in i.get('datestring'):
            i['datestring']=i.get('datestring')+[str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:]]
    i['allyears']=sorted(i.get('allyears'), reverse=True)
    i['datestring']=sorted(i.get('datestring'), reverse=True)
    upperlist=set()
    for j in sum(i.get('target'),[]):
        upperlist.add(j.upper())
    i['targetshapes']=list(upperlist)
    i['addtime']=now_time

In [112]:
Gene_all=copy.deepcopy(Gene_output_json_final)

# RNA_raw

In [113]:
collections_RNA_raw=db.RNA_raw

In [114]:
names=locals()

In [115]:
raw_pick=pickle.load(open('results/step1/raw_pick.pkl','rb'))

In [116]:
raw_pick_pubtator=pickle.load(open('results/step1/raw_pick_pubtator.pkl','rb'))

In [117]:
pubtator_raw=pickle.load(open('results/step1/pubtator_raw.pkl','rb'))

In [118]:
pubtator_gene=[]
for i in pubtator_raw:
    if i.get('type')=='Gene':
        pubtator_gene.append(i)

In [119]:
os.makedirs('../microRNA_lncRNA_rules/'+now_time)

In [120]:
with open('../microRNA_lncRNA_rules/'+now_time+'/allsentences.txt','a') as f:
    for i in raw_pick_pubtator:
        f.write(i.get('AB')+'\n'+'\n')

In [121]:
with open('../microRNA_lncRNA_rules/'+now_time+'/run.sh','w') as f:
    f.write('''
#!/bin/bash\n
start_time=$(date +%s)
java -cp "../stanza_corenlp/*" -Xmx500g edu.stanford.nlp.pipeline.StanfordCoreNLP -annotators 'tokenize,ssplit,regexner' -file "../microRNA_lncRNA_rules/'''
+now_time+'''/allsentences.txt" -regexner.mapping ../microRNA_lncRNA_rules/jg-regexner.txt -regexner.ignorecase true -tokenize.options "splitHyphenated=false" --outputFormat json --outputDirectory ../microRNA_lncRNA_rules/'''+now_time+'''/  
end_time=$(date +%s)
cost_time=$[ $end_time-$start_time ]
echo "build kernel time is $(($cost_time/60))min $(($cost_time%60))s"
    ''')

In [123]:
os.system("bash ../microRNA_lncRNA_rules/"+now_time+"/run.sh > ../microRNA_lncRNA_rules/"+now_time+"/run.log")

[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator tokenize
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ssplit
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator regexner
[main] INFO edu.stanford.nlp.pipeline.TokensRegexNERAnnotator - regexner: Read 21 unique entries out of 22 from ../microRNA_lncRNA_rules/jg-regexner.txt, 0 TokensRegex patterns.

Processing file /root/hald_autoupdate/2023_05_09/../microRNA_lncRNA_rules/2023_05_09/allsentences.txt ... writing to /root/hald_autoupdate/microRNA_lncRNA_rules/2023_05_09/allsentences.txt.json
Annotating file /root/hald_autoupdate/2023_05_09/../microRNA_lncRNA_rules/2023_05_09/allsentences.txt ... done [65.7 sec].

Annotation pipeline timing information:
TokenizerAnnotator: 1.7 sec.
WordsToSentencesAnnotator: 0.7 sec.
TokensRegexNERAnnotator: 63.3 sec.
TOTAL: 65.7 sec. for 417253 tokens at 6353.6 tokens/sec.
Pipeline setup: 0.6 sec.
Total time for StanfordCoreNLP pipeli

0

In [124]:
jsonfile={}
json_file=json.load(open('../microRNA_lncRNA_rules/'+now_time+'/allsentences.txt.json',"r"))
jsonfile["sentences"]=json_file.get("sentences")

In [125]:
words=[]
entities_dict_list=[]
starttime=datetime.datetime.now()
l=0
HALDnames=[]
start=0
for HALDname in ["LNCRNA","MICRORNA"]:
    HALDnames.append("HALD_"+HALDname)
f=open('../microRNA_lncRNA_rules/'+now_time+'/allsentences.txt')
sentences=f.read()
for i in jsonfile.get("sentences"):
    for HALDname in HALDnames:
        judge=True
        size=len(i.get("tokens"))
        for k,j in enumerate(i.get("tokens")):
            if j.get("ner"):
                if j.get("ner")==HALDname:
                    combination=j.get('word')
                    words.append(combination)
                    entities_dict={}
                    entities_dict["word"]=combination
                    entities_dict["wordBegin"]=j.get("characterOffsetBegin")+start
                    entities_dict["wordEnd"]=j.get("characterOffsetEnd")+start
                    entities_dict["sentenceBegin"]=i.get("tokens")[0].get("characterOffsetBegin")+start
                    entities_dict["sentenceEnd"]=i.get("tokens")[-1].get("characterOffsetEnd")+start
                    entities_dict["sentence"]=sentences[i.get("tokens")[0].get("characterOffsetBegin"):i.get("tokens")[-1].get("characterOffsetEnd")]
                    entities_dict["ner"]=HALDname
                    entities_dict_list.append(entities_dict)
                    if judge:
                        names['l'+HALDname]=k+1
                        judge=False
                    if names['l'+HALDname]<k:
                        names['l'+HALDname]=k+1
                    if k != names['l'+HALDname] -1:
                        continue
                    if names['l'+HALDname]<size:
                        if i.get("tokens")[names['l'+HALDname]].get("ner"):
                            if i.get("tokens")[names['l'+HALDname]].get("ner")==HALDname:
                                while i.get("tokens")[names['l'+HALDname]].get("ner")==HALDname:
                                    combination+=" "+i.get("tokens")[names['l'+HALDname]].get("word")
                                    names['l'+HALDname]+=1
                                    if names['l'+HALDname]>=size:
                                        break
                                names['l'+HALDname]+=1
                                words.append(combination)
                                entities_dict={}
                                entities_dict["word"]=combination
                                entities_dict["wordBegin"]=j.get("characterOffsetBegin")+start
                                entities_dict["wordEnd"]=j.get("characterOffsetEnd")+start
                                entities_dict["sentenceBegin"]=i.get("tokens")[0].get("characterOffsetBegin")+start
                                entities_dict["sentenceEnd"]=i.get("tokens")[-1].get("characterOffsetEnd")+start
                                entities_dict["sentence"]=sentences[i.get("tokens")[0].get("characterOffsetBegin"):i.get("tokens")[-1].get("characterOffsetEnd")]
                                entities_dict["ner"]=HALDname
                                entities_dict_list.append(entities_dict)
f.close()
start+=len(sentences)
endtime=datetime.datetime.now()
print("The running time is "+str((endtime-starttime).seconds)+"s")

The running time is 0s


In [126]:
len(entities_dict_list)

58

In [127]:
entities_dict_list[2]

{'word': 'miR-16-5p',
 'wordBegin': 516917,
 'wordEnd': 516926,
 'sentenceBegin': 516909,
 'sentenceEnd': 517030,
 'sentence': 'In ECs, miR-16-5p, miR-451, miR-191-5p, miR-27a-3p, and miR-27b-3p, in that order, were the top detectable miRNAs in ECs.',
 'ner': 'HALD_MICRORNA'}

In [128]:
length2pmid=[]
start=0
for i in raw_pick_pubtator:
    mid=[]
    mid.append(start)
    mid.append(i.get('PMID'))
    length2pmid.append(mid)
    start+=len(i.get('AB'))+2

In [129]:
len(length2pmid)

1402

In [130]:
length2pmid[10]

[16170, '37058445']

In [131]:
k=0
for i in entities_dict_list:
    for j in range(k,len(length2pmid)):
        if i.get('wordBegin')<length2pmid[j][0]:
            i['PMID']=length2pmid[j-1][1]
            i['AB']=pubtator_raw_dict2AB.get(i.get('PMID'))
            k=j-1
            break
    else:
        i['PMID']=length2pmid[-1][1]
        i['AB']=pubtator_raw_dict2AB.get(i.get('PMID'))

In [132]:
entities_dict_list[10]

{'word': 'miR-128-3p',
 'wordBegin': 518130,
 'wordEnd': 518140,
 'sentenceBegin': 517955,
 'sentenceEnd': 518287,
 'sentence': 'Remarkably, the observed SIV-mediated decrease in miR-128-3p levels in EVs from RMs agrees with publicly available EV miRNAome data by Kaddour et al., 2021, which showed that miR-128-3p levels were significantly lower in semen-derived EVs from HIV-infected men who used or did not use cocaine compared to HIV-uninfected individuals.',
 'ner': 'HALD_MICRORNA',
 'PMID': '36992331',
 'AB': 'Background: This is Manuscript 1 of a two-part Manuscript of the same series. Here, we present findings from our first set of studies on the abundance and compartmentalization of blood plasma extracellular microRNAs (exmiRNAs) into extracellular particles, including blood plasma extracellular vesicles (EVs) and extracellular condensates (ECs) in the setting of untreated HIV/SIV infection. The goals of the study presented in this Manuscript 1 are to (i) assess the abundance and 

In [134]:
RNA2ncbi=pd.read_csv('../ncbi/RNA_ncbi.txt',sep='\t',encoding='utf-8',converters = {'NCBI Gene ID(supplied by NCBI)':str})

In [135]:
RNA2approved_dict=dict(zip(RNA2ncbi['NCBI Gene ID(supplied by NCBI)'],RNA2ncbi['Approved symbol'] ))

In [136]:
RNA2ncbi_dict=dict(zip(RNA2ncbi['Approved symbol'], RNA2ncbi['NCBI Gene ID(supplied by NCBI)']))

In [137]:
error_RNA=[]
pubtator_RNAs=[]
for i in pubtator_gene:
    if RNA2approved_dict.get(i.get('identifier')):
        i['entity']=RNA2approved_dict.get(i.get('identifier'))
        pubtator_RNAs.append(i)
    else:
        error_RNA.append(i)

In [138]:
len(error_RNA)

2124

In [139]:
len(pubtator_RNAs)

28

In [140]:
names=locals()
HALDnames=[]
for HALDname in ["CLUSTERRNA","LNCRNA","MICRORNA","MISCRNA","RIBOSOMALRNA","SMALL_NUCLEARRNA","SMALL_NUCLEOLARRNA","TRANSFERRNA","VAULTRNA","YRNA"]:
    HALDnames.append("HALD_"+HALDname)
for name in ["clusterRNA","lncRNA","microRNA","miscRNA","ribosomalRNA","small_nuclearRNA","small_nucleolarRNA","transferRNA","vaultRNA","yRNA"]:
    file="../dict/"+name+"/"+name+"dict.json"
    with open(file,"r") as f:
        jsons=json.load(f)
        json_lower={}
        for i,j in jsons.items():
            json_lower[i.lower()]=j
        names[name+"_dict"]=json_lower

In [141]:
pattern=['([Hh][Ss][Aa])?[\-]?[Mm][Ii][Cc][Rr][Oo][Rr][Nn][Aa][\-]?\d+[a-zA-Z]?[\-]?[12345]?[\-]?[35]?[Pp]?',
'([Hh][Ss][Aa])?[\-]?[Mm][Ii][Cc][Rr][Oo][Rr][Nn][Aa] \d+[a-zA-Z]?[\-]?[12345]?[\-]?[35]?[Pp]?',
'([Hh][Ss][Aa])?[\-]?[Mm][Ii][Rr][Nn]?[\-]?\d+[a-zA-Z]?[\-]?[12345]?[\-]?[35]?[Pp]?',
'([Hh][Ss][Aa])?[\-]?[Mm][Ii][Rr][Nn]? \d+[a-zA-Z]?[\-]?[12345]?[\-]?[35]?[Pp]?',
'([Hh][Ss][Aa])?[\-]?([Mm][Ii][Cc][Rr][Oo][Rr][Nn][Aa][\-]?\d+)([a-z][/]?)+',
'([Hh][Ss][Aa])?[\-]?[Mm][Ii][Cc][Rr][Oo][Rr][Nn][Aa] \d+([a-z][/]?)+',
'([Hh][Ss][Aa])?[\-]?([Mm][Ii][Rr][Nn]?[\-]?\d+)([a-z][/]?)+',
'([Hh][Ss][Aa])?[\-]?[Mm][Ii][Rr][Nn]? \d+([a-z][/]?)+',
'[Hh][Ss][Aa][\-][Mm][Ii][Rr][Nn]?[\-]\d+[a-zA-Z]?[\-]?[12345]?[\-]?[35]?[Pp]?',
'([Hh][Ss][Aa][\-][Mm][Ii][Rr][Nn]?[\-]\d+)([a-z][/]?)+',
'[Mm][Ii][Rr][Nn]?[Ll][Ee][Tt]7[A-Ia-i][123]?',
'([Mm][Ii][Rr][Nn]?[Ll][Ee][Tt]7)([A-Ia-i][/]?)+',
'[Mm][Ii][Cc][Rr][Oo][Rr][Nn][Aa][\-][Ll][Ee][Tt][\-]\d+[a-zA-Z]?[\-]?[12345]?[\-]?[35]?[Pp]?',
'[Mm][Ii][Cc][Rr][Oo][Rr][Nn][Aa] [Ll][Ee][Tt] \d+[a-zA-Z]?[\-]?[12345]?[\-]?[35]?[Pp]?',
'[Mm][Ii][Rr][Nn]?[\-][Ll][Ee][Tt][\-]\d+[a-zA-Z]?[\-]?[12345]?[\-]?[35]?[Pp]?',
'[Mm][Ii][Rr][Nn]? [Ll][Ee][Tt] \d+[a-zA-Z]?[\-]?[12345]?[\-]?[35]?[Pp]?',
'([Mm][Ii][Cc][Rr][Oo][Rr][Nn][Aa][\-][Ll][Ee][Tt][\-]\d+)([a-z][/]?)+',
'[Mm][Ii][Cc][Rr][Oo][Rr][Nn][Aa] [Ll][Ee][Tt] \d+([a-z][/]?)+',
'([Mm][Ii][Rr][Nn]?[\-][Ll][Ee][Tt][\-]\d+)([a-z][/]?)+',
'[Mm][Ii][Rr][Nn]? [Ll][Ee][Tt] \d+([a-z][/]?)+']

In [142]:
filtered_pubtator_RNAs=[]
drop_RNAs=[]
midcount1=0
midcount2=0
for i in pubtator_RNAs:
    for name in ["clusterRNA","lncRNA","microRNA","miscRNA","ribosomalRNA","small_nuclearRNA","small_nucleolarRNA","transferRNA","vaultRNA","yRNA"]:
        if names[name+"_dict"].get(i.get('text').lower()):
            i['ner']="HALD_"+name.upper()
            i['type']="RNA"
            i['specific_type']=name.upper()
            filtered_pubtator_RNAs.append(i)
            break
    else:
        for j in pattern:
            if re.search(j,i.get('text').lower()):
                midcount1+=1
                i['ner']="HALD_MICRORNA"
                i['type']="RNA"
                i['specific_type']="MICRORNA"
                filtered_pubtator_RNAs.append(i)
                break
            elif re.search(r'linc\d{5}',i.get('text').lower()):
                midcount2+=1
                i['ner']="HALD_LINCRNA"
                i['type']="RNA"
                i['specific_type']="LINCRNA"
                filtered_pubtator_RNAs.append(i)
                break
        else:
            drop_RNAs.append(i)

In [143]:
print(midcount1)
print(midcount2)

9
0


In [144]:
len(drop_RNAs)

2

In [145]:
filtered_pubtator_genes_pmid2word={}
for i in filtered_pubtator_RNAs:
    filtered_pubtator_genes_pmid2word[i.get('id')]=filtered_pubtator_genes_pmid2word.get(i.get('id'),[])+[i.get('text')]

In [146]:
len(filtered_pubtator_genes_pmid2word)

9

In [147]:
# based on dict. Whether in pubtator
RNA_entities_list=[]
for i in entities_dict_list:
    if filtered_pubtator_genes_pmid2word.get(i.get('PMID')):
        if i.get('word') not in filtered_pubtator_genes_pmid2word.get(i.get('PMID')):
            RNA_entities_list.append(i)
    else:
        RNA_entities_list.append(i)

In [148]:
len(RNA_entities_list)

35

In [149]:
len(entities_dict_list)

58

In [150]:
RNA_entities_list[11]

{'word': 'miR-29',
 'wordBegin': 897914,
 'wordEnd': 897920,
 'sentenceBegin': 897872,
 'sentenceEnd': 898000,
 'sentence': 'The results displayed an up-regulation of miR-29 and miR-96 and a down-regulation of miR-146 in EVs derived from elderly donors.',
 'ner': 'HALD_MICRORNA',
 'PMID': '36921675',
 'AB': 'Blood donor age has become a major concern due to the age-associated variations in the content and concentration of circulating extracellular nano-sized vesicles (EVs), including exosomes. These EVs mirror the state of their parental cells and transfer it to the recipient cells via biological messengers such as microRNAs (miRNAs, miRs). Since the behavior of hematopoietic stem cells (HSCs) is potentially affected by the miRs of plasma-derived EVs, a better understanding of the content of EVs is important for the safety and efficacy perspectives in blood transfusion medicine. Herein, we investigated whether the plasma-derived EVs of young (18-25 years) and elderly human donors (45-6

In [151]:
for i in RNA_entities_list:
    i['type']='RNA'
    i['specific_type']=i.get('ner')[5:]

In [152]:
RNAdict=json.load(open('../dict/RNAdict.json','r'))

In [153]:
RNAdict_lower={}
for i,j in RNAdict.items():
    RNAdict_lower[i.lower()]=RNAdict_lower.get(i.lower(),[])+j
    RNAdict_lower[i.lower()]=list(set(RNAdict_lower.get(i.lower())))

In [154]:
cccc=0
final_RNA_entities_list=[]
drop_RNA_entities_list=[]
for i in RNA_entities_list:
    if RNAdict_lower.get(i.get('word').lower()):
        for j in RNAdict_lower.get(i.get('word').lower()):
            i['entity']=j
            final_RNA_entities_list.append(i)
    elif re.search(r'-[3,5]p',i.get('word')):
            if RNAdict_lower.get(re.sub(r'-[3,5]p','',i.get('word'))):
                for j in RNAdict_lower.get(re.sub(r'-[3,5]p','',i.get('word'))):
                    i['entity']=j
                    final_RNA_entities_list.append(i)
                    cccc+=1
    else:
        drop_RNA_entities_list.append(i)

In [155]:
len(final_RNA_entities_list)

7

In [156]:
len(drop_RNA_entities_list)

8

In [157]:
drop_RNA_entities_list[0]

{'word': 'miR-128',
 'wordBegin': 518364,
 'wordEnd': 518371,
 'sentenceBegin': 518288,
 'sentenceEnd': 518399,
 'sentence': 'These findings confirmed our previously reported finding and suggested that miR-128 may be a target of HIV/SIV.',
 'ner': 'HALD_MICRORNA',
 'PMID': '36992331',
 'AB': 'Background: This is Manuscript 1 of a two-part Manuscript of the same series. Here, we present findings from our first set of studies on the abundance and compartmentalization of blood plasma extracellular microRNAs (exmiRNAs) into extracellular particles, including blood plasma extracellular vesicles (EVs) and extracellular condensates (ECs) in the setting of untreated HIV/SIV infection. The goals of the study presented in this Manuscript 1 are to (i) assess the abundance and compartmentalization of exmiRNAs in EVs versus ECs in the healthy uninfected state, and (ii) evaluate how SIV infection may affect exmiRNA abundance and compartmentalization in these particles. Considerable effort has been d

In [158]:
for i in filtered_pubtator_RNAs:
    i['PMID']=i.get('id')
    i['word']=i.get('text')

In [159]:
final_RNA=filtered_pubtator_RNAs+final_RNA_entities_list

In [160]:
len(final_RNA)

33

In [161]:
RNAs_inte_copy=copy.deepcopy(final_RNA)

In [162]:
deduplicated_RNA=set()
deduplicated_RNA_count=0
RNA_list_raw=[]
for i in RNAs_inte_copy:
    k=i.get('PMID')
    i['DP']=rawpickwithAB.get(k).get('DP')
    i['AB']=rawpickwithAB.get(k).get('AB')
    i['JT']=rawpickwithAB.get(k).get('JT')
    i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+k
    i['TI']=rawpickwithAB.get(k).get('TI')
    i['target']=i.get('word')
    tuples=(i.get("PMID"),i.get("entity"),i.get("word"),i.get("sentence"))
    if tuples not in deduplicated_RNA:
        deduplicated_RNA.add(tuples)
        RNA_list_raw.append(i)
    else:
        deduplicated_RNA_count+=1
        continue

In [163]:
len(RNA_list_raw)

32

In [164]:
RNA2blank={}
for i in RNA_list_raw:
    RNA2blank[i.get('entity')]=''

In [165]:
len(RNA2blank)

12

In [166]:
RNA_converter=pickle.load(open('../ncbi/RNA_converter.pkl','rb'))

In [167]:
len(RNA_converter)

344

In [168]:
def get_RNA_info(i,final_RNAs_inte):
    ncbiid=RNA2ncbi_dict.get(i)
    ncbiurl='https://www.ncbi.nlm.nih.gov/gene/' + ncbiid
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"
    }
    url = 'https://www.ncbi.nlm.nih.gov/gene/'+ncbiid
    res=requests.get(url=url,headers=headers)
    html = etree.HTML(res.content)
    ncbisummary={
    'Official_Symbol' : html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Official")]/following::dd[1]/text()')[0],
    'Official_Full_Name' : html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Official")]/following::dd[1]/text()')[1],
    'HGNC_ID' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Primary source")]/following::dd[1]/a/text()')),
    'HGNC_URL' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Primary source")]/following::dd[1]/a/@href')),
    'Ensembl_ID' :  ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[1]/text()')),
    'Ensembl_URL' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[1]/@href')),
    'MIM_ID' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[2]/text()')),
    'MIM_URL' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[2]/@href')),
    'AllianceGenome_ID' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[3]/text()')),
    'AllianceGenome_URL' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[3]/@href')),
    'NCBI_ID' : ncbiid ,
    'NCBI_URL' : ncbiurl
    }
    try:
        also=''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Also known as")]/following::dd[1]/text()')).replace('HGNC:HGNC:','')
    except:
        also=''
    finally:
        ncbisummary.update({'Also_known_as' : also})
    try:
        also2=''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Summary")]/following::dd[1]/text()'))
    except:
        also2=''
    finally:
        ncbisummary.update({'Summary' : also2})
    final_RNAs_inte[i]=ncbisummary
    time.sleep(0.5)

In [169]:
judge=False
p = Pool(10)
manager=Manager()
final_RNAs_inte = manager.dict()
for i in RNA2blank.keys():
    if not RNA_converter.get(i):
        p.apply_async(get_RNA_info, args=(i,final_RNAs_inte))
        judge=True
p.close()
p.join()
if judge:
    RNA_converter.update(dict(final_RNAs_inte))
    pickle.dump(RNA_converter, open('../ncbi/RNA_converter.pkl','wb'))

In [170]:
len(RNA_converter)

350

In [171]:
for i in range(len(RNA_list_raw)):
    RNA_list_raw[i]=dict(RNA_list_raw[i],**RNA_converter.get(RNA_list_raw[i].get('entity')))

In [172]:
dropout=[]
months=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
for i in RNA_list_raw:
    j=i.get('DP')
    if re.search('[S,s]upplement',j,flags=re.M|re.I):
        j=re.sub('[S,s]upplement.*','',j,flags=re.M|re.I)
    j=re.sub('\s?-\s?','-',j)
    j=re.sub('\.','',j)
    j=re.sub('Aout','Aug',j)
    if len(j.split(' '))==3:
        try:
            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
        except:
            if j.split()[1] in months:
                if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                    j=j.split()[0]+' '+j.split()[1]+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                        print('***************************************ERROR**********************************************')
                else:
                    j=j.split()[0]+' '+j.split()[1]+' '+ '1'
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        print(j)
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
            elif j.split()[1] not in months:
                if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                        middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                    else:
                        middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if middle in months:
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+middle+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+middle+' '+ '1'
                    elif re.match(r'Sum(mer)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Jun'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Jun'+' '+ '1'
                    elif re.match(r'Spr(ing)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Mar'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Mar'+' '+ '1'
                    elif re.match(r'Win(ter)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Dec'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Dec'+' '+ '1'    
                    elif re.match(r'Aut(umn)?',middle) or \
                    re.match(r'Fall',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Sep'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Sep'+' '+ '1'      
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        print(j)
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                else:
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    elif len(j.split())==2:
        if re.match(r'Sum(mer)?',j.split()[1]):
            date=int(j[:4]+'0601')
        elif re.match(r'Spr(ing)?',j.split()[1]):
            date=int(j[:4]+'0301')
        elif re.match(r'Win(ter)?',j.split()[1]):
            date=int(j[:4]+'1201')
        elif re.match(r'Aut(umn)?',j.split()[1]) or re.match(r'Fall',j.split()[1]):
            date=int(j[:4]+'0901')
        elif re.match(r'Sum(mer)?',j.split()[0]):
            date=int(j[-4:]+'0601')
        elif re.match(r'Spr(ing)?',j.split()[0]):
            date=int(j[-4:]+'0301')
        elif re.match(r'Win(ter)?',j.split()[0]):
            date=int(j[-4:]+'1201')
        elif re.match(r'Aut(umn)?',j.split()[0]) or re.match(r'Fall',j.split()[0]):
            date=int(j[-4:]+'0901')
        else:
            try:
                date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b'), '%Y%m'+'01'))
            except:
                if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                        middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                    else:
                        middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if middle in months:
                        j=j.split()[0]+' '+middle+' '+ '1'
                    elif re.match(r'Sum(mer)?',middle):
                        j=j.split()[0]+' '+'Jun'+' '+ '1'
                    elif re.match(r'Spr(ing)?',middle):
                        j=j.split()[0]+' '+'Mar'+' '+ '1'
                    elif re.match(r'Win(ter)?',middle):
                        j=j.split()[0]+' '+'Dec'+' '+ '1'    
                    elif re.match(r'Aut(umn)?',middle) or \
                    re.match(r'Fall',middle):
                        j=j.split()[0]+' '+'Sep'+' '+ '1'      
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                elif re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    midd=re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if len(midd)!=2:
                        midd='0'+midd
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+midd+'01'))
                else:
                    print(j)
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    elif len(j.split())==1:
        j=j[:4]
        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    else:
        j=j.split()[0]+' '+j.split()[1]+' '+re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
    i['date']=date
    i['year']=int(str(i.get('date'))[:4])

In [173]:
RNA_list_raw[0]

{'id': '36992331',
 'AB': 'Background: This is Manuscript 1 of a two-part Manuscript of the same series. Here, we present findings from our first set of studies on the abundance and compartmentalization of blood plasma extracellular microRNAs (exmiRNAs) into extracellular particles, including blood plasma extracellular vesicles (EVs) and extracellular condensates (ECs) in the setting of untreated HIV/SIV infection. The goals of the study presented in this Manuscript 1 are to (i) assess the abundance and compartmentalization of exmiRNAs in EVs versus ECs in the healthy uninfected state, and (ii) evaluate how SIV infection may affect exmiRNA abundance and compartmentalization in these particles. Considerable effort has been devoted to studying the epigenetic control of viral infection, particularly in understanding the role of exmiRNAs as key regulators of viral pathogenesis. MicroRNA (miRNAs) are small (~20-22 nts) non-coding RNAs that regulate cellular processes through targeted mRNA d

In [174]:
for i in RNA_list_raw:
    i['addtime']=now_time

In [175]:
for i,j in RNA_converter.items():
    if j.get('Summary') and j.get('Summary')[0]=='\n':
        print(i)

In [176]:
if len(RNA_list_raw)!=0:
    collections_RNA_raw.insert_many(RNA_list_raw)

# RNA_all

In [177]:
RNA_raw_list=list(collections_RNA_raw.find())

In [178]:
for i in RNA_raw_list:
    i['type']='RNA'
    i["PMID"]=[i.get("PMID")]
    i["sentence"]=[[i.get("sentence")]]
    pmid=i.get('PMID')[0]
    i['DP']=[i.get("DP")]
    i['AB']=[i.get("AB")]
    i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+pmid
    i['TI']=[i.get("TI")]
    i['target']=[[i.get('word')]]
    i["date"]=[i.get("date")]
    i["year"]=[i.get("year")]
    i['JT']=[i.get('JT')]

In [179]:
RNA_mid={}
for i in RNA_raw_list:
    if not RNA_mid.get(i.get("entity")):
        RNA_mid[i.get("entity")]=i
    else:
        if i.get('PMID')[0] in RNA_mid.get(i.get('entity')).get('PMID'):
            ind=RNA_mid.get(i.get('entity')).get('PMID').index(i.get('PMID')[0])
            if i.get("sentence")[0][0] in RNA_mid.get(i.get('entity')).get('sentence')[ind]:
                continue
            else:
                RNA_mid[i.get('entity')]['sentence'][ind] = RNA_mid.get(i.get('entity')).get('sentence')[ind] + i.get("sentence")[0]
                RNA_mid[i.get('entity')]['target'][ind] = RNA_mid.get(i.get('entity')).get('target')[ind] + i.get("target")[0]
        else:
            RNA_mid[i.get("entity")]["PMID"]=RNA_mid.get(i.get("entity")).get("PMID")+i.get("PMID")
            RNA_mid[i.get("entity")]["sentence"]=RNA_mid.get(i.get("entity")).get("sentence")+i.get("sentence")
            RNA_mid[i.get("entity")]["AB"]=RNA_mid.get(i.get("entity")).get("AB")+i.get("AB")
            RNA_mid[i.get("entity")]["DP"]=RNA_mid.get(i.get("entity")).get("DP")+i.get("DP")
            RNA_mid[i.get("entity")]["TI"]=RNA_mid.get(i.get("entity")).get("TI")+i.get("TI")
            RNA_mid[i.get("entity")]["target"]=RNA_mid.get(i.get("entity")).get("target")+i.get("target")
            RNA_mid[i.get("entity")]["date"]=RNA_mid.get(i.get("entity")).get("date")+i.get("date")
            RNA_mid[i.get("entity")]["year"]=RNA_mid.get(i.get("entity")).get("year")+i.get("year")
            RNA_mid[i.get("entity")]["JT"]=RNA_mid.get(i.get("entity")).get("JT")+i.get("JT")

In [180]:
RNA_output_json_final=list(RNA_mid.values())

In [181]:
len(RNA_output_json_final)

350

In [182]:
for i in RNA_output_json_final:
    i['allyears']=[]
    i['datestring']=[]
    for j in i.get('date'):
        if int(str(j)[:4]) not in i.get('allyears'):
            i['allyears']= i.get('allyears')+[int(str(j)[:4])]
    for j in i.get('date'):
        if str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:] not in i.get('datestring'):
            i['datestring']=i.get('datestring')+[str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:]]
    i['allyears']=sorted(i.get('allyears'), reverse=True)
    i['datestring']=sorted(i.get('datestring'), reverse=True)
    upperlist=set()
    for j in sum(i.get('target'),[]):
        upperlist.add(j.upper())
    i['targetshapes']=list(upperlist)
    i['addtime']=now_time

In [183]:
RNA_all=copy.deepcopy(RNA_output_json_final)

# chemical_raw

In [184]:
chemical_pubtator_addpmid={}
for i in pubtator_raw:
    if i.get('type')=='Chemical':
        if chemical_pubtator_addpmid.get(i.get('id')):
            chemical_pubtator_addpmid[i.get('id')]=chemical_pubtator_addpmid.get(i.get('id'))+[i]
        else:
            chemical_pubtator_addpmid[i.get('id')]=[i]

In [185]:
print(len(chemical_pubtator_addpmid))

538


In [186]:
chemical_output_jsons=list(chemical_pubtator_addpmid.values())

In [187]:
chemical_output_json=[]
for i in chemical_output_jsons:
    for j in i:
        chemical_output_json.append(j)

In [188]:
chemical_raw=copy.deepcopy(chemical_output_json)

In [190]:
pairs=['Toxin','Peptide','Lipid','Protein','Carbohydrate','Pharmaceutical Preparations']

In [191]:
names=locals()

In [192]:
alllist=[]
for j in pairs:
    names[j.replace(' ','_').lower()]=pickle.load(open('../chemical/'+j.replace(' ','_')+'/'+j.replace(' ','_')+'.pkl','rb'))
    alllist+=names[j.replace(' ','_').lower()][1:]

In [193]:
alllist[0]

{'text': 'Bacterial Toxins',
 'url': '/mesh/68001427',
 'MeSHUniqueID': 'D001427',
 'type': 'Toxin'}

In [194]:
for i in alllist:
    if not i.get('MeSHUniqueID'):
        i['MeSHUniqueID']='D'+i.get('url')[-6:]

In [195]:
chemicallist=[]
MESHlist=[]
notchemicallist=[]
MESHurl=[]
for i in alllist:
    if i.get('MeSHUniqueID'):
        chemicallist.append({'MESH':'MESH:'+i.get('MeSHUniqueID'),'type':i.get('type'),'url':i.get('url'),'text':i.get('text')})
        MESHlist.append('MESH:'+i.get('MeSHUniqueID'))
        MESHurl.append(i.get('url'))
    else:
        notchemicallist.append(i)

In [196]:
len(notchemicallist)

0

In [197]:
chemical2classdict={}
count=0
for i in chemicallist:
    if not chemical2classdict.get(i.get('MESH')):
        chemical2classdict[i.get('MESH')]=[i.get('type')]
    else:
        if i.get('type') in chemical2classdict.get(i.get('MESH')):
            pass
        else:
            chemical2classdict[i.get('MESH')]=chemical2classdict.get(i.get('MESH'))+[i.get('type')]
            count+=1

In [198]:
chemical2subclassdict={}
for i in chemicallist:
    chemical2subclassdict[i.get('MESH')]=i.get('text')

In [199]:
chemical2urldict={}
for i in chemicallist:
    chemical2urldict[i.get('MESH')]=i.get('url')

In [200]:
for i in chemical_raw:
    i['PMID']=i.get('id')
    i['word']=i.get('text')
    del i['id']
    del i['text']

In [201]:
chemical_final=[]
chemical_mid=[]
chemical_dropout=[]

In [202]:
multi_chemical_type={}
for i in chemical_raw:
    if chemical2classdict.get(i.get('identifier')):
        chemical_final.append(dict(i,**{'class':chemical2classdict.get(i.get('identifier')),'subclass':chemical2subclassdict.get(i.get('identifier')),'url':chemical2urldict.get(i.get('identifier'))}))
        for j in chemical2classdict.get(i.get('identifier')):
            if len(chemical2classdict.get(i.get('identifier')))>1:
                if multi_chemical_type.get(chemical2subclassdict.get(i.get('identifier'))):
                    if j not in multi_chemical_type.get(chemical2subclassdict.get(i.get('identifier'))):
                        multi_chemical_type[chemical2subclassdict.get(i.get('identifier'))]=multi_chemical_type.get(chemical2subclassdict.get(i.get('identifier')))+[j]
                else:
                    multi_chemical_type[chemical2subclassdict.get(i.get('identifier'))]=[j]
    else:
        chemical_dropout.append(i)

In [203]:
len(chemical_final)

255

In [204]:
multi_chemical_type

{'Arginine Vasopressin': ['Peptide', 'Protein'],
 'Bleomycin': ['Peptide', 'Carbohydrate']}

In [205]:
len(chemical_dropout)

2268

In [206]:
for j in pairs:
    names[j.replace(' ','_')+'list']=[]
    for i in chemical_final:
        if j in i.get('class'):
            names[j.replace(' ','_')+'list'].append(i)
    print(len(names[j.replace(' ','_')+'list']))

2
33
81
18
123
1


In [207]:
chemical_deduplicated=set()
chemical_deduplicated_count=0
for j in pairs:
    names[j.replace(' ','_')+'list_raw']=[]
    for i in names[j.replace(' ','_')+'list']:
        k=i.get('PMID')
        i['DP']=rawpickwithAB.get(k).get('DP')
        i['AB']=rawpickwithAB.get(k).get('AB')
        i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+k
        i['TI']=rawpickwithAB.get(k).get('TI')
        i['entity']=i.get('subclass')
        i['target']=i.get('word')
        i['type']=j
        i['JT']=rawpickwithAB.get(k).get('JT')
        if i.get('identifier') and re.match(r'MESH:',i.get('identifier')):
            i['MESH']=i.get('identifier')[5:]
        else:
            i['MESH']=''
        tuples=(i.get("PMID"),i.get("entity"),i.get("word"),i.get("sentence"),i.get("type"))
        if tuples not in chemical_deduplicated:
            chemical_deduplicated.add(tuples)
            names[j.replace(' ','_')+'list_raw'].append(i)
        else:
            chemical_deduplicated_count+=1
            continue

In [208]:
chemical2ncbi={}
c=0
for j in pairs:
    for i in names[j.replace(' ','_')+'list_raw']:
        chemical2ncbi[i.get('url')]=''
        c+=1

In [209]:
len(chemical2ncbi)

53

In [210]:
chemical_converter=pickle.load(open('../ncbi/Chemical_converter.pkl','rb'))

In [211]:
len(chemical_converter)

484

In [212]:
def get_chemical_info(i,final_chemicals_inte):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"
    }
    url = 'https://www.ncbi.nlm.nih.gov' + i
    res=requests.get(url=url,headers=headers)
    html = etree.HTML(res.content)
    ncbisummary={
    'Summary' : ''.join(html.xpath('//*[@class="mesh_ds_scope_note"]/text()')),
    'YearIntroduced' : ''.join(html.xpath('//*[@class="mesh_year_introduced"]/text()'))[17:21],
    }
    final_chemicals_inte.update({i:ncbisummary})
    time.sleep(0.5)

In [213]:
judge=False
p = Pool(10)
manager=Manager()
final_chemicals_inte = manager.dict()
for i in chemical2ncbi.keys():
    if not chemical_converter.get(i):
        p.apply_async(get_chemical_info, args=(i,final_chemicals_inte))
        judge=True
p.close()
p.join()
if judge:
    chemical_converter.update(dict(final_chemicals_inte))
    pickle.dump(chemical_converter, open('../ncbi/Chemical_converter.pkl','wb'))

In [214]:
len(chemical_converter)

488

In [215]:
pairs

['Toxin',
 'Peptide',
 'Lipid',
 'Protein',
 'Carbohydrate',
 'Pharmaceutical Preparations']

In [216]:
for j in pairs:
    for i in range(len(names[j.replace(' ','_')+'list_raw'])):
        names[j.replace(' ','_')+'list_raw'][i]=dict(names[j.replace(' ','_')+'list_raw'][i],**chemical_converter.get(names[j.replace(' ','_')+'list_raw'][i].get('url')))

In [217]:
dropout=[]
months=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
for k in pairs:
    for i in names[k.replace(' ','_')+'list_raw']:
        j=i.get('DP')
        if re.search('[S,s]upplement',j,flags=re.M|re.I):
            j=re.sub('[S,s]upplement.*','',j,flags=re.M|re.I)
        j=re.sub('\s?-\s?','-',j)
        j=re.sub('\.','',j)
        j=re.sub('Aout','Aug',j)
        if len(j.split(' '))==3:
            try:
                date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
            except:
                if j.split()[1] in months:
                    if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                        j=j.split()[0]+' '+j.split()[1]+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        try:
                            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                        except:
                            j=j[:4]
                            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                            print('***************************************ERROR**********************************************')
                    else:
                        j=j.split()[0]+' '+j.split()[1]+' '+ '1'
                        try:
                            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                        except:
                            print(j)
                            j=j[:4]
                            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                elif j.split()[1] not in months:
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                        if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                            middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                        else:
                            middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                        if middle in months:
                            if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                                j=j.split()[0]+' '+middle+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                            else:
                                j=j.split()[0]+' '+middle+' '+ '1'
                        elif re.match(r'Sum(mer)?',middle):
                            if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                                j=j.split()[0]+' '+'Jun'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                            else:
                                j=j.split()[0]+' '+'Jun'+' '+ '1'
                        elif re.match(r'Spr(ing)?',middle):
                            if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                                j=j.split()[0]+' '+'Mar'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                            else:
                                j=j.split()[0]+' '+'Mar'+' '+ '1'
                        elif re.match(r'Win(ter)?',middle):
                            if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                                j=j.split()[0]+' '+'Dec'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                            else:
                                j=j.split()[0]+' '+'Dec'+' '+ '1'    
                        elif re.match(r'Aut(umn)?',middle) or \
                        re.match(r'Fall',middle):
                            if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                                j=j.split()[0]+' '+'Sep'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                            else:
                                j=j.split()[0]+' '+'Sep'+' '+ '1'      
                        try:
                            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                        except:
                            print(j)
                            j=j[:4]
                            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                    else:
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
        elif len(j.split())==2:
            if re.match(r'Sum(mer)?',j.split()[1]):
                date=int(j[:4]+'0601')
            elif re.match(r'Spr(ing)?',j.split()[1]):
                date=int(j[:4]+'0301')
            elif re.match(r'Win(ter)?',j.split()[1]):
                date=int(j[:4]+'1201')
            elif re.match(r'Aut(umn)?',j.split()[1]) or re.match(r'Fall',j.split()[1]):
                date=int(j[:4]+'0901')
            elif re.match(r'Sum(mer)?',j.split()[0]):
                date=int(j[-4:]+'0601')
            elif re.match(r'Spr(ing)?',j.split()[0]):
                date=int(j[-4:]+'0301')
            elif re.match(r'Win(ter)?',j.split()[0]):
                date=int(j[-4:]+'1201')
            elif re.match(r'Aut(umn)?',j.split()[0]) or re.match(r'Fall',j.split()[0]):
                date=int(j[-4:]+'0901')
            else:
                try:
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b'), '%Y%m'+'01'))
                except:
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                        if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                            middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                        else:
                            middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                        if middle in months:
                            j=j.split()[0]+' '+middle+' '+ '1'
                        elif re.match(r'Sum(mer)?',middle):
                            j=j.split()[0]+' '+'Jun'+' '+ '1'
                        elif re.match(r'Spr(ing)?',middle):
                            j=j.split()[0]+' '+'Mar'+' '+ '1'
                        elif re.match(r'Win(ter)?',middle):
                            j=j.split()[0]+' '+'Dec'+' '+ '1'    
                        elif re.match(r'Aut(umn)?',middle) or \
                        re.match(r'Fall',middle):
                            j=j.split()[0]+' '+'Sep'+' '+ '1'      
                        try:
                            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                        except:
                            j=j[:4]
                            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                    elif re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                        midd=re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                        if len(midd)!=2:
                            midd='0'+midd
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+midd+'01'))
                    else:
                        print(j)
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
        elif len(j.split())==1:
            j=j[:4]
            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
        else:
            j=j.split()[0]+' '+j.split()[1]+' '+re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
        i['date']=date
        i['year']=int(str(i.get('date'))[:4])

In [218]:
for k in pairs:
    names[k.replace(' ','_')+'_raw']=copy.deepcopy(names[k.replace(' ','_')+'list_raw'])
    for i in names[k.replace(' ','_')+'_raw']:
        i['addtime']=now_time

# Carbohydrate_raw

In [219]:
collections_Carbohydrate_raw=db.Carbohydrate_raw

In [220]:
len(Carbohydrate_raw)

109

In [221]:
Carbohydrate_raw[0]

{'AB': 'Panax ginseng C. A. Meyer (ginseng), a traditional Chinese herb, is usually used to improve health and increase anti-aging activity for human. Polysaccharides are bioactive components of ginseng. Herein, using Caenorhabditis elegans as a model, we discovered a ginseng-derived rhamnogalacturonan I (RG-I) pectin WGPA-1-RG promoted longevity via TOR signalling pathway with transcription factors FOXO/DAF-16 and Nrf2/SKN-1 accumulated in the nucleus, where they activated target genes. And the WGPA-1-RG-mediated lifespan extension was dependent on endocytosis, rather than a bacterial metabolic process. Glycosidic linkage analyses combined with arabinose- and galactose-releasing enzyme hydrolyses identified the RG-I backbone of WGPA-1-RG was primarily substituted with alpha-1,5-linked arabinan, beta-1,4-linked galactan and arabinogalactan II (AG-II) side chains. Feeding worms with the WGPA-1-RG-derived fractions which lost distinct structural elements by enzymatic digestions, we found

In [222]:
if len(Carbohydrate_raw)!=0:
    collections_Carbohydrate_raw.insert_many(Carbohydrate_raw)

In [223]:
shenme=list(collections_Carbohydrate_raw.find())

In [224]:
shenme[-1]

{'_id': ObjectId('6459f29748e9c8c06012be44'),
 'AB': 'CLINICAL RELEVANCE: Hand hygiene is important to reduce the spread of microbes in clinical settings. Hand sanitisers that last longer may be beneficial. BACKGROUND: Longevity of hand sanitisation products on fingers and hands may be important to help reduce microbial transmission. The current study evaluated the persistence of disinfection of three hand sanitisers. METHODS: Initially the minimum inhibitory concentrations of the hand sanitisers were determined using strains of Staphylococcus epidermidis and S. aureus. Then a cross-over study with participants randomly assigned to use three different hand sanitisers for 30 seconds was undertaken. The number of bacteria and fungi on fingers was assessed 10 and 20 minutes and 4 hours after use. The type of microbial inhibition of the capric acid sanitiser was studied by examining the effects of adding Tween 80 and lecithin to microbial agar. RESULTS: The minimum inhibitory concentration

# Carbohydrate_all

In [225]:
Carbohydrate_raw_all=list(collections_Carbohydrate_raw.find())

In [226]:
len(Carbohydrate_raw_all)

17738

In [227]:
for i in Carbohydrate_raw_all:
    i['type']='Carbohydrate'
    i["PMID"]=[i.get("PMID")]
    i["sentence"]=[[i.get("sentence")]]
    pmid=i.get('PMID')[0]
    i['DP']=[i.get("DP")]
    i['AB']=[i.get("AB")]
    i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+pmid
    i['TI']=[i.get("TI")]
    i['target']=[[i.get('word')]]
    i["date"]=[i.get("date")]
    i["year"]=[i.get("year")]
    i['JT']=[i.get('JT')]

In [228]:
Carbohydrate_mid={}
for i in Carbohydrate_raw_all:
    if not Carbohydrate_mid.get(i.get("entity")):
        Carbohydrate_mid[i.get("entity")]=i
    else:
        if i.get('PMID')[0] in Carbohydrate_mid.get(i.get('entity')).get('PMID'):
            ind=Carbohydrate_mid.get(i.get('entity')).get('PMID').index(i.get('PMID')[0])
            if i.get("sentence")[0][0] in Carbohydrate_mid.get(i.get('entity')).get('sentence')[ind]:
                continue
            else:
                Carbohydrate_mid[i.get('entity')]['sentence'][ind] = Carbohydrate_mid.get(i.get('entity')).get('sentence')[ind] + i.get("sentence")[0]
                Carbohydrate_mid[i.get('entity')]['target'][ind] = Carbohydrate_mid.get(i.get('entity')).get('target')[ind] + i.get("target")[0]
        else:
            Carbohydrate_mid[i.get("entity")]["PMID"]=Carbohydrate_mid.get(i.get("entity")).get("PMID")+i.get("PMID")
            Carbohydrate_mid[i.get("entity")]["sentence"]=Carbohydrate_mid.get(i.get("entity")).get("sentence")+i.get("sentence")
            Carbohydrate_mid[i.get("entity")]["AB"]=Carbohydrate_mid.get(i.get("entity")).get("AB")+i.get("AB")
            Carbohydrate_mid[i.get("entity")]["DP"]=Carbohydrate_mid.get(i.get("entity")).get("DP")+i.get("DP")
            Carbohydrate_mid[i.get("entity")]["TI"]=Carbohydrate_mid.get(i.get("entity")).get("TI")+i.get("TI")
            Carbohydrate_mid[i.get("entity")]["target"]=Carbohydrate_mid.get(i.get("entity")).get("target")+i.get("target")
            Carbohydrate_mid[i.get("entity")]["date"]=Carbohydrate_mid.get(i.get("entity")).get("date")+i.get("date")
            Carbohydrate_mid[i.get("entity")]["year"]=Carbohydrate_mid.get(i.get("entity")).get("year")+i.get("year")
            Carbohydrate_mid[i.get("entity")]["JT"]=Carbohydrate_mid.get(i.get("entity")).get("JT")+i.get("JT")

In [229]:
Carbohydrate_all=list(Carbohydrate_mid.values())

In [230]:
len(Carbohydrate_all)

204

In [231]:
for i in Carbohydrate_all:
    i['allyears']=[]
    i['datestring']=[]
    for j in i.get('date'):
        if int(str(j)[:4]) not in i.get('allyears'):
            i['allyears']= i.get('allyears')+[int(str(j)[:4])]
    for j in i.get('date'):
        if str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:] not in i.get('datestring'):
            i['datestring']=i.get('datestring')+[str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:]]
    i['allyears']=sorted(i.get('allyears'), reverse=True)
    i['datestring']=sorted(i.get('datestring'), reverse=True)
    upperlist=set()
    for j in sum(i.get('target'),[]):
        upperlist.add(j.upper())
    i['targetshapes']=list(upperlist)
    i['addtime']=now_time

In [232]:
Carbohydrate_all[10].get('Summary')

'A class of sphingolipids found largely in the brain and other nervous tissue. They contain phosphocholine or phosphoethanolamine as their polar head group so therefore are the only sphingolipids classified as PHOSPHOLIPIDS.'

# Lipid_raw

In [234]:
collections_Lipid_raw=db.Lipid_raw

In [235]:
len(Lipid_raw)

72

In [236]:
if len(Lipid_raw)!=0:
    collections_Lipid_raw.insert_many(Lipid_raw)

# Lipid_all

In [237]:
Lipid_raw_all=list(collections_Lipid_raw.find())

In [238]:
len(Lipid_raw_all)

19204

In [239]:
for i in Lipid_raw_all:
    if i.get('entity')=='Angiotensins':
        print(i)

In [240]:
for i in Lipid_raw_all:
    i['type']='Lipid'
    i["PMID"]=[i.get("PMID")]
    i["sentence"]=[[i.get("sentence")]]
    pmid=i.get('PMID')[0]
    i['DP']=[i.get("DP")]
    i['AB']=[i.get("AB")]
    i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+pmid
    i['TI']=[i.get("TI")]
    i['target']=[[i.get('word')]]
    i["date"]=[i.get("date")]
    i["year"]=[i.get("year")]
    i['JT']=[i.get('JT')]

In [241]:
Lipid_mid={}
for i in Lipid_raw_all:
    if not Lipid_mid.get(i.get("entity")):
        Lipid_mid[i.get("entity")]=i
    else:
        if i.get('PMID')[0] in Lipid_mid.get(i.get('entity')).get('PMID'):
            ind=Lipid_mid.get(i.get('entity')).get('PMID').index(i.get('PMID')[0])
            if i.get("sentence")[0][0] in Lipid_mid.get(i.get('entity')).get('sentence')[ind]:
                continue
            else:
                Lipid_mid[i.get('entity')]['sentence'][ind] = Lipid_mid.get(i.get('entity')).get('sentence')[ind] + i.get("sentence")[0]
                Lipid_mid[i.get('entity')]['target'][ind] = Lipid_mid.get(i.get('entity')).get('target')[ind] + i.get("target")[0]
        else:
            Lipid_mid[i.get("entity")]["PMID"]=Lipid_mid.get(i.get("entity")).get("PMID")+i.get("PMID")
            Lipid_mid[i.get("entity")]["sentence"]=Lipid_mid.get(i.get("entity")).get("sentence")+i.get("sentence")
            Lipid_mid[i.get("entity")]["AB"]=Lipid_mid.get(i.get("entity")).get("AB")+i.get("AB")
            Lipid_mid[i.get("entity")]["DP"]=Lipid_mid.get(i.get("entity")).get("DP")+i.get("DP")
            Lipid_mid[i.get("entity")]["TI"]=Lipid_mid.get(i.get("entity")).get("TI")+i.get("TI")
            Lipid_mid[i.get("entity")]["target"]=Lipid_mid.get(i.get("entity")).get("target")+i.get("target")
            Lipid_mid[i.get("entity")]["date"]=Lipid_mid.get(i.get("entity")).get("date")+i.get("date")
            Lipid_mid[i.get("entity")]["year"]=Lipid_mid.get(i.get("entity")).get("year")+i.get("year")
            Lipid_mid[i.get("entity")]["JT"]=Lipid_mid.get(i.get("entity")).get("JT")+i.get("JT")

In [242]:
Lipid_all=list(Lipid_mid.values())

In [243]:
len(Lipid_all)

191

In [244]:
for i in Lipid_all:
    i['allyears']=[]
    i['datestring']=[]
    for j in i.get('date'):
        if int(str(j)[:4]) not in i.get('allyears'):
            i['allyears']= i.get('allyears')+[int(str(j)[:4])]
    for j in i.get('date'):
        if str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:] not in i.get('datestring'):
            i['datestring']=i.get('datestring')+[str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:]]
    i['allyears']=sorted(i.get('allyears'), reverse=True)
    i['datestring']=sorted(i.get('datestring'), reverse=True)
    upperlist=set()
    for j in sum(i.get('target'),[]):
        upperlist.add(j.upper())
    i['targetshapes']=list(upperlist)
    i['addtime']=now_time

# Peptide_raw

In [246]:
collections_Peptide_raw=db.Peptide_raw

In [247]:
len(Peptide_raw)

31

In [248]:
if len(Peptide_raw)!=0:
    collections_Peptide_raw.insert_many(Peptide_raw)

# Peptide_all

In [250]:
Peptide_raw_all=list(collections_Peptide_raw.find())

In [251]:
len(Peptide_raw_all)

3443

In [252]:
for i in Peptide_raw_all:
    if i.get('entity')=='Angiotensins':
        print(i)
        break

{'_id': ObjectId('64005023648b16758fce3875'), 'AB': 'AIMS: Real-world evidence about heart failure (HF) medical care for elderly patients is limited. The aim of this study was to clarify the age-dependent associations of HF medications at discharge with clinical outcomes in a super-aged society using a real-world big database. METHODS AND RESULTS: Patients hospitalized with a first episode of acute HF in 2013-2019 were identified in a nationwide claims database in Japan and were analyzed based on the HF medications at discharge. Hazard ratios (HRs) and 95 % confidence intervals (95 % CIs) for clinical outcomes were calculated after adjustment. Among 325,468 patients from 4351 hospitals, 130,230 (40.0 %) were aged >= 85 years. All-cause mortality and HF readmission rate within 1 year were 14 % and 23 %, respectively. beta-Blockers and angiotensin-converting-enzyme inhibitors or angiotensin II receptor blockers (ACEI/ARB) uses were associated with lower mortality (HR [95 %CI]: 0.84 [0.83

In [253]:
for i in Peptide_raw_all:
    i['type']='Peptide'
    i["PMID"]=[i.get("PMID")]
    i["sentence"]=[[i.get("sentence")]]
    pmid=i.get('PMID')[0]
    i['DP']=[i.get("DP")]
    i['AB']=[i.get("AB")]
    i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+pmid
    i['TI']=[i.get("TI")]
    i['target']=[[i.get('word')]]
    i["date"]=[i.get("date")]
    i["year"]=[i.get("year")]
    i['JT']=[i.get('JT')]

In [254]:
Peptide_mid={}
for i in Peptide_raw_all:
    if not Peptide_mid.get(i.get("entity")):
        Peptide_mid[i.get("entity")]=i
    else:
        if i.get('PMID')[0] in Peptide_mid.get(i.get('entity')).get('PMID'):

            ind=Peptide_mid.get(i.get('entity')).get('PMID').index(i.get('PMID')[0])
            if i.get("sentence")[0][0] in Peptide_mid.get(i.get('entity')).get('sentence')[ind]:
                continue
            else:
                Peptide_mid[i.get('entity')]['sentence'][ind] = Peptide_mid.get(i.get('entity')).get('sentence')[ind] + i.get("sentence")[0]
                Peptide_mid[i.get('entity')]['target'][ind] = Peptide_mid.get(i.get('entity')).get('target')[ind] + i.get("target")[0]

        else:
            Peptide_mid[i.get("entity")]["PMID"]=Peptide_mid.get(i.get("entity")).get("PMID")+i.get("PMID")
            Peptide_mid[i.get("entity")]["sentence"]=Peptide_mid.get(i.get("entity")).get("sentence")+i.get("sentence")
            Peptide_mid[i.get("entity")]["AB"]=Peptide_mid.get(i.get("entity")).get("AB")+i.get("AB")
            Peptide_mid[i.get("entity")]["DP"]=Peptide_mid.get(i.get("entity")).get("DP")+i.get("DP")
            Peptide_mid[i.get("entity")]["TI"]=Peptide_mid.get(i.get("entity")).get("TI")+i.get("TI")
            Peptide_mid[i.get("entity")]["target"]=Peptide_mid.get(i.get("entity")).get("target")+i.get("target")
            Peptide_mid[i.get("entity")]["date"]=Peptide_mid.get(i.get("entity")).get("date")+i.get("date")
            Peptide_mid[i.get("entity")]["year"]=Peptide_mid.get(i.get("entity")).get("year")+i.get("year")
            Peptide_mid[i.get("entity")]["JT"]=Peptide_mid.get(i.get("entity")).get("JT")+i.get("JT")

In [255]:
Peptide_all=list(Peptide_mid.values())

In [256]:
len(Peptide_all)

79

In [257]:
for i in Peptide_all:
    i['allyears']=[]
    i['datestring']=[]
    for j in i.get('date'):
        if int(str(j)[:4]) not in i.get('allyears'):
            i['allyears']= i.get('allyears')+[int(str(j)[:4])]
    for j in i.get('date'):
        if str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:] not in i.get('datestring'):
            i['datestring']=i.get('datestring')+[str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:]]
    i['allyears']=sorted(i.get('allyears'), reverse=True)
    i['datestring']=sorted(i.get('datestring'), reverse=True)
    upperlist=set()
    for j in sum(i.get('target'),[]):
        upperlist.add(j.upper())
    i['targetshapes']=list(upperlist)
    i['addtime']=now_time

# Pharmaceutical_Preparations_raw

In [259]:
collections_Pharmaceutical_Preparations_raw=db.Pharmaceutical_Preparations_raw

In [260]:
len(Pharmaceutical_Preparations_raw)

1

In [261]:
if len(Pharmaceutical_Preparations_raw)!=0:
    collections_Pharmaceutical_Preparations_raw.insert_many(Pharmaceutical_Preparations_raw)

# Pharmaceutical_Preparations_all

In [263]:
Pharmaceutical_Preparations_raw_all=list(collections_Pharmaceutical_Preparations_raw.find())

In [264]:
len(Pharmaceutical_Preparations_raw_all)

270

In [265]:
for i in Pharmaceutical_Preparations_raw_all:
    i['type']='Pharmaceutical Preparations'
    i["PMID"]=[i.get("PMID")]
    i["sentence"]=[[i.get("sentence")]]
    pmid=i.get('PMID')[0]
    i['DP']=[i.get("DP")]
    i['AB']=[i.get("AB")]
    i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+pmid
    i['TI']=[i.get("TI")]
    i['target']=[[i.get('word')]]
    i["date"]=[i.get("date")]
    i["year"]=[i.get("year")]
    i['JT']=[i.get('JT')]

In [266]:
Pharmaceutical_Preparations_mid={}
for i in Pharmaceutical_Preparations_raw_all:
    if not Pharmaceutical_Preparations_mid.get(i.get("entity")):
        Pharmaceutical_Preparations_mid[i.get("entity")]=i
    else:
        if i.get('PMID')[0] in Pharmaceutical_Preparations_mid.get(i.get('entity')).get('PMID'):
            ind=Pharmaceutical_Preparations_mid.get(i.get('entity')).get('PMID').index(i.get('PMID')[0])
            if i.get("sentence")[0][0] in Pharmaceutical_Preparations_mid.get(i.get('entity')).get('sentence')[ind]:
                continue
            else:
                Pharmaceutical_Preparations_mid[i.get('entity')]['sentence'][ind] = Pharmaceutical_Preparations_mid.get(i.get('entity')).get('sentence')[ind] + i.get("sentence")[0]
                Pharmaceutical_Preparations_mid[i.get('entity')]['target'][ind] = Pharmaceutical_Preparations_mid.get(i.get('entity')).get('target')[ind] + i.get("target")[0]
        else:
            Pharmaceutical_Preparations_mid[i.get("entity")]["PMID"]=Pharmaceutical_Preparations_mid.get(i.get("entity")).get("PMID")+i.get("PMID")
            Pharmaceutical_Preparations_mid[i.get("entity")]["sentence"]=Pharmaceutical_Preparations_mid.get(i.get("entity")).get("sentence")+i.get("sentence")
            Pharmaceutical_Preparations_mid[i.get("entity")]["AB"]=Pharmaceutical_Preparations_mid.get(i.get("entity")).get("AB")+i.get("AB")
            Pharmaceutical_Preparations_mid[i.get("entity")]["DP"]=Pharmaceutical_Preparations_mid.get(i.get("entity")).get("DP")+i.get("DP")
            Pharmaceutical_Preparations_mid[i.get("entity")]["TI"]=Pharmaceutical_Preparations_mid.get(i.get("entity")).get("TI")+i.get("TI")
            Pharmaceutical_Preparations_mid[i.get("entity")]["target"]=Pharmaceutical_Preparations_mid.get(i.get("entity")).get("target")+i.get("target")
            Pharmaceutical_Preparations_mid[i.get("entity")]["date"]=Pharmaceutical_Preparations_mid.get(i.get("entity")).get("date")+i.get("date")
            Pharmaceutical_Preparations_mid[i.get("entity")]["year"]=Pharmaceutical_Preparations_mid.get(i.get("entity")).get("year")+i.get("year")
            Pharmaceutical_Preparations_mid[i.get("entity")]["JT"]=Pharmaceutical_Preparations_mid.get(i.get("entity")).get("JT")+i.get("JT")

In [267]:
Pharmaceutical_Preparations_all=list(Pharmaceutical_Preparations_mid.values())

In [268]:
len(Pharmaceutical_Preparations_all)

11

In [269]:
for i in Pharmaceutical_Preparations_all:
    i['allyears']=[]
    i['datestring']=[]
    for j in i.get('date'):
        if int(str(j)[:4]) not in i.get('allyears'):
            i['allyears']= i.get('allyears')+[int(str(j)[:4])]
    for j in i.get('date'):
        if str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:] not in i.get('datestring'):
            i['datestring']=i.get('datestring')+[str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:]]
    i['allyears']=sorted(i.get('allyears'), reverse=True)
    i['datestring']=sorted(i.get('datestring'), reverse=True)
    upperlist=set()
    for j in sum(i.get('target'),[]):
        upperlist.add(j.upper())
    i['targetshapes']=list(upperlist)
    i['addtime']=now_time

# Protein_raw

In [271]:
collections_Protein_raw=db.Protein_raw

In [272]:
len(Protein_raw)

18

In [273]:
if len(Protein_raw)!=0:
    collections_Protein_raw.insert_many(Protein_raw)

# Protein_all

In [275]:
Protein_raw_all=list(collections_Protein_raw.find())

In [276]:
len(Protein_raw_all)

1875

In [277]:
for i in Protein_raw_all:
    if i.get('entity')=='Angiotensins':
        print(i)

{'_id': ObjectId('6400506e648b16758fce4078'), 'AB': 'AIMS: Real-world evidence about heart failure (HF) medical care for elderly patients is limited. The aim of this study was to clarify the age-dependent associations of HF medications at discharge with clinical outcomes in a super-aged society using a real-world big database. METHODS AND RESULTS: Patients hospitalized with a first episode of acute HF in 2013-2019 were identified in a nationwide claims database in Japan and were analyzed based on the HF medications at discharge. Hazard ratios (HRs) and 95 % confidence intervals (95 % CIs) for clinical outcomes were calculated after adjustment. Among 325,468 patients from 4351 hospitals, 130,230 (40.0 %) were aged >= 85 years. All-cause mortality and HF readmission rate within 1 year were 14 % and 23 %, respectively. beta-Blockers and angiotensin-converting-enzyme inhibitors or angiotensin II receptor blockers (ACEI/ARB) uses were associated with lower mortality (HR [95 %CI]: 0.84 [0.83

In [278]:
for i in Protein_raw_all:
    i['type']='Protein'
    i["PMID"]=[i.get("PMID")]
    i["sentence"]=[[i.get("sentence")]]
    pmid=i.get('PMID')[0]
    i['DP']=[i.get("DP")]
    i['AB']=[i.get("AB")]
    i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+pmid
    i['TI']=[i.get("TI")]
    i['target']=[[i.get('word')]]
    i["date"]=[i.get("date")]
    i["year"]=[i.get("year")]
    i['JT']=[i.get('JT')]

In [279]:
Protein_mid={}
for i in Protein_raw_all:
    if not Protein_mid.get(i.get("entity")):
        Protein_mid[i.get("entity")]=i
    else:
        if i.get('PMID')[0] in Protein_mid.get(i.get('entity')).get('PMID'):
            ind=Protein_mid.get(i.get('entity')).get('PMID').index(i.get('PMID')[0])
            if i.get("sentence")[0][0] in Protein_mid.get(i.get('entity')).get('sentence')[ind]:
                continue
            else:
                Protein_mid[i.get('entity')]['sentence'][ind] = Protein_mid.get(i.get('entity')).get('sentence')[ind] + i.get("sentence")[0]
                Protein_mid[i.get('entity')]['target'][ind] = Protein_mid.get(i.get('entity')).get('target')[ind] + i.get("target")[0]
        else:
            Protein_mid[i.get("entity")]["PMID"]=Protein_mid.get(i.get("entity")).get("PMID")+i.get("PMID")
            Protein_mid[i.get("entity")]["sentence"]=Protein_mid.get(i.get("entity")).get("sentence")+i.get("sentence")
            Protein_mid[i.get("entity")]["AB"]=Protein_mid.get(i.get("entity")).get("AB")+i.get("AB")
            Protein_mid[i.get("entity")]["DP"]=Protein_mid.get(i.get("entity")).get("DP")+i.get("DP")
            Protein_mid[i.get("entity")]["TI"]=Protein_mid.get(i.get("entity")).get("TI")+i.get("TI")
            Protein_mid[i.get("entity")]["target"]=Protein_mid.get(i.get("entity")).get("target")+i.get("target")
            Protein_mid[i.get("entity")]["date"]=Protein_mid.get(i.get("entity")).get("date")+i.get("date")
            Protein_mid[i.get("entity")]["year"]=Protein_mid.get(i.get("entity")).get("year")+i.get("year")
            Protein_mid[i.get("entity")]["JT"]=Protein_mid.get(i.get("entity")).get("JT")+i.get("JT")

In [280]:
Protein_all=list(Protein_mid.values())

In [281]:
len(Protein_all)

31

In [282]:
for i in Protein_all:
    i['allyears']=[]
    i['datestring']=[]
    for j in i.get('date'):
        if int(str(j)[:4]) not in i.get('allyears'):
            i['allyears']= i.get('allyears')+[int(str(j)[:4])]
    for j in i.get('date'):
        if str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:] not in i.get('datestring'):
            i['datestring']=i.get('datestring')+[str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:]]
    i['allyears']=sorted(i.get('allyears'), reverse=True)
    i['datestring']=sorted(i.get('datestring'), reverse=True)
    upperlist=set()
    for j in sum(i.get('target'),[]):
        upperlist.add(j.upper())
    i['targetshapes']=list(upperlist)
    i['addtime']=now_time

# Toxin_raw

In [284]:
collections_Toxin_raw=db.Toxin_raw

In [285]:
len(Toxin_raw)

2

In [286]:
if len(Toxin_raw)!=0:
    collections_Toxin_raw.insert_many(Toxin_raw)

# Toxin_all

In [288]:
Toxin_raw_all=list(collections_Toxin_raw.find())

In [289]:
len(Toxin_raw_all)

542

In [290]:
for i in Toxin_raw_all:
    i['type']='Toxin'
    i["PMID"]=[i.get("PMID")]
    i["sentence"]=[[i.get("sentence")]]
    pmid=i.get('PMID')[0]
    i['DP']=[i.get("DP")]
    i['AB']=[i.get("AB")]
    i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+pmid
    i['TI']=[i.get("TI")]
    i['target']=[[i.get('word')]]
    i["date"]=[i.get("date")]
    i["year"]=[i.get("year")]
    i['JT']=[i.get('JT')]

In [291]:
Toxin_mid={}
for i in Toxin_raw_all:
    if not Toxin_mid.get(i.get("entity")):
        Toxin_mid[i.get("entity")]=i
    else:
        if i.get('PMID')[0] in Toxin_mid.get(i.get('entity')).get('PMID'):
            ind=Toxin_mid.get(i.get('entity')).get('PMID').index(i.get('PMID')[0])
            if i.get("sentence")[0][0] in Toxin_mid.get(i.get('entity')).get('sentence')[ind]:
                continue
            else:
                Toxin_mid[i.get('entity')]['sentence'][ind] = Toxin_mid.get(i.get('entity')).get('sentence')[ind] + i.get("sentence")[0]
                Toxin_mid[i.get('entity')]['target'][ind] = Toxin_mid.get(i.get('entity')).get('target')[ind] + i.get("target")[0]
        else:
            Toxin_mid[i.get("entity")]["PMID"]=Toxin_mid.get(i.get("entity")).get("PMID")+i.get("PMID")
            Toxin_mid[i.get("entity")]["sentence"]=Toxin_mid.get(i.get("entity")).get("sentence")+i.get("sentence")
            Toxin_mid[i.get("entity")]["AB"]=Toxin_mid.get(i.get("entity")).get("AB")+i.get("AB")
            Toxin_mid[i.get("entity")]["DP"]=Toxin_mid.get(i.get("entity")).get("DP")+i.get("DP")
            Toxin_mid[i.get("entity")]["TI"]=Toxin_mid.get(i.get("entity")).get("TI")+i.get("TI")
            Toxin_mid[i.get("entity")]["target"]=Toxin_mid.get(i.get("entity")).get("target")+i.get("target")
            Toxin_mid[i.get("entity")]["date"]=Toxin_mid.get(i.get("entity")).get("date")+i.get("date")
            Toxin_mid[i.get("entity")]["year"]=Toxin_mid.get(i.get("entity")).get("year")+i.get("year")
            Toxin_mid[i.get("entity")]["JT"]=Toxin_mid.get(i.get("entity")).get("JT")+i.get("JT")

In [292]:
Toxin_all=list(Toxin_mid.values())

In [293]:
len(Toxin_all)

19

In [294]:
for i in Toxin_all:
    i['allyears']=[]
    i['datestring']=[]
    for j in i.get('date'):
        if int(str(j)[:4]) not in i.get('allyears'):
            i['allyears']= i.get('allyears')+[int(str(j)[:4])]
    for j in i.get('date'):
        if str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:] not in i.get('datestring'):
            i['datestring']=i.get('datestring')+[str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:]]
    i['allyears']=sorted(i.get('allyears'), reverse=True)
    i['datestring']=sorted(i.get('datestring'), reverse=True)
    upperlist=set()
    for j in sum(i.get('target'),[]):
        upperlist.add(j.upper())
    i['targetshapes']=list(upperlist)
    i['addtime']=now_time

# Mutation_raw

In [296]:
collections_Mutation_raw=db.Mutation_raw

In [297]:
Mutation_pubtator_addpmid={}
for i in pubtator_raw:
    if i.get('type')=='Mutation':
        if Mutation_pubtator_addpmid.get(i.get('id')):
            Mutation_pubtator_addpmid[i.get('id')]=Mutation_pubtator_addpmid.get(i.get('id'))+[i]
        else:
            Mutation_pubtator_addpmid[i.get('id')]=[i]

In [298]:
Mutation_output_json=list(Mutation_pubtator_addpmid.values())

In [299]:
Mutation_raw=sum(Mutation_output_json,[])

In [300]:
len(Mutation_raw)

38

In [301]:
for i in Mutation_raw:
    i['PMID']=i.get('id')
    i['word']=i.get('text')
    del i['id']
    del i['text']

In [302]:
Mutation_deduplicated=set()
Mutation_deduplicated_count=0
mutation_list_raw=[]
for i in Mutation_raw:
    if i.get('identifier'):
        k=i.get('PMID')
        i['DP']=rawpickwithAB.get(k).get('DP')
        i['AB']=rawpickwithAB.get(k).get('AB')
        i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+k
        i['TI']=rawpickwithAB.get(k).get('TI')
        i['entity']=i.get('identifier')
        i['target']=i.get('word')
        i['type']='Mutation'
        i['JT']=rawpickwithAB.get(k).get('JT')
        tuples=(i.get("PMID"),i.get("entity"),i.get("word"),i.get("sentence"))
        if tuples not in Mutation_deduplicated:
            Mutation_deduplicated.add(tuples)
            mutation_list_raw.append(i)
        else:
            Mutation_deduplicated_count+=1
            continue

In [303]:
len(mutation_list_raw)

34

In [304]:
dropout1=[]
dropout2=[]
for i in mutation_list_raw:
    if re.match(r'rs\d+',i.get('identifier')):
        dropout1.append(i)
    else:
        dropout2.append(i)

In [305]:
len(dropout1)

28

In [306]:
len(dropout2)

6

In [307]:
mutation2ncbi={}
for i in dropout1:
    mutation2ncbi[i.get('entity')]={}

In [308]:
len(mutation2ncbi)

13

In [309]:
mutation_converter=pickle.load(open('../ncbi/Mutation_converter.pkl','rb'))

In [310]:
len(mutation_converter)

1893

In [311]:
def get_mutation_info(i,final_mutations_inte):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"
    }
    url = 'https://www.ncbi.nlm.nih.gov/snp/' + i
    html = etree.HTML(requests.get(url=url,headers=headers).content)
    ncbisummary={
    'Position':''.join(html.xpath('//*[@class="usa-width-one-half"][1]/dd[2]/span[1]/text()'))+''.join(html.xpath('//*[@class="usa-width-one-half"][1]/dd[2]/span[2]/text()')),
    'Alleles':''.join(html.xpath('normalize-space(//*[@class="usa-width-one-half"][1]/dd[3]/text())')),
    'Clinical Significance':'https://www.ncbi.nlm.nih.gov'+''.join(html.xpath('//*[@class="usa-width-one-half"][2]/dd[1]/a/@href')) if html.xpath('//*[@class="usa-width-one-half"][2]/dd[1]/a/@href') else '',
    'Gene : Consequence':[''.join(html.xpath('//*[@class="usa-width-one-half"][2]/dd[2]/div[1]/text()')),''.join(html.xpath('//*[@class="usa-width-one-half"][2]/dd[2]/span[1]/text()'))] if ''.join(html.xpath('//*[@class="usa-width-one-half"][2]/dd[2]/div[1]/text()')) else [''.join(html.xpath('//*[@class="usa-width-one-half"][2]/dd[2]/span[1]/text()'))],
    'Genomic View':url+''.join(html.xpath('//*[@class="usa-width-one-half"][2]/dd[4]/a/@href')),
    }
    final_mutations_inte.update({i:ncbisummary})
    time.sleep(0.5)

In [312]:
judge=False
p = Pool(10)
manager=Manager()
final_mutations_inte = manager.dict()
for i in mutation2ncbi.keys():
    if not mutation_converter.get(i):
        p.apply_async(get_mutation_info, args=(i,final_mutations_inte))
        judge=True
p.close()
p.join()
if judge:
    mutation_converter.update(dict(final_mutations_inte))
    pickle.dump(mutation_converter, open('../ncbi/Mutation_converter.pkl','wb'))

In [313]:
len(mutation_converter)

1902

In [314]:
for i in range(len(mutation_list_raw)):
    if re.match(r'rs\d+',mutation_list_raw[i].get('entity')):
        mutation_list_raw[i]=dict(mutation_list_raw[i],**mutation_converter.get(mutation_list_raw[i].get('entity')))
    else:
        mutation_list_raw[i]=dict(mutation_list_raw[i],**{'Position':'','Alleles':'','Clinical Significance':'','Gene : Consequence':[''],'Genomic View':''})

In [315]:
dropout=[]
months=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
for i in mutation_list_raw:
    j=i.get('DP')
    if re.search('[S,s]upplement',j,flags=re.M|re.I):
        j=re.sub('[S,s]upplement.*','',j,flags=re.M|re.I)
    j=re.sub('\s?-\s?','-',j)
    j=re.sub('\.','',j)
    j=re.sub('Aout','Aug',j)
    if len(j.split(' '))==3:
        try:
            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
        except:
            if j.split()[1] in months:
                if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                    j=j.split()[0]+' '+j.split()[1]+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                        print('***************************************ERROR**********************************************')
                else:
                    j=j.split()[0]+' '+j.split()[1]+' '+ '1'
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        print(j)
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
            elif j.split()[1] not in months:
                if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                        middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                    else:
                        middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if middle in months:
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+middle+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+middle+' '+ '1'
                    elif re.match(r'Sum(mer)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Jun'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Jun'+' '+ '1'
                    elif re.match(r'Spr(ing)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Mar'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Mar'+' '+ '1'
                    elif re.match(r'Win(ter)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Dec'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Dec'+' '+ '1'    
                    elif re.match(r'Aut(umn)?',middle) or \
                    re.match(r'Fall',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Sep'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Sep'+' '+ '1'      
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        print(j)
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                else:
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    elif len(j.split())==2:
        if re.match(r'Sum(mer)?',j.split()[1]):
            date=int(j[:4]+'0601')
        elif re.match(r'Spr(ing)?',j.split()[1]):
            date=int(j[:4]+'0301')
        elif re.match(r'Win(ter)?',j.split()[1]):
            date=int(j[:4]+'1201')
        elif re.match(r'Aut(umn)?',j.split()[1]) or re.match(r'Fall',j.split()[1]):
            date=int(j[:4]+'0901')
        elif re.match(r'Sum(mer)?',j.split()[0]):
            date=int(j[-4:]+'0601')
        elif re.match(r'Spr(ing)?',j.split()[0]):
            date=int(j[-4:]+'0301')
        elif re.match(r'Win(ter)?',j.split()[0]):
            date=int(j[-4:]+'1201')
        elif re.match(r'Aut(umn)?',j.split()[0]) or re.match(r'Fall',j.split()[0]):
            date=int(j[-4:]+'0901')
        else:
            try:
                date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b'), '%Y%m'+'01'))
            except:
                if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                        middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                    else:
                        middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if middle in months:
                        j=j.split()[0]+' '+middle+' '+ '1'
                    elif re.match(r'Sum(mer)?',middle):
                        j=j.split()[0]+' '+'Jun'+' '+ '1'
                    elif re.match(r'Spr(ing)?',middle):
                        j=j.split()[0]+' '+'Mar'+' '+ '1'
                    elif re.match(r'Win(ter)?',middle):
                        j=j.split()[0]+' '+'Dec'+' '+ '1'    
                    elif re.match(r'Aut(umn)?',middle) or \
                    re.match(r'Fall',middle):
                        j=j.split()[0]+' '+'Sep'+' '+ '1'      
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                elif re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    midd=re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if len(midd)!=2:
                        midd='0'+midd
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+midd+'01'))
                else:
                    print(j)
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    elif len(j.split())==1:
        j=j[:4]
        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    else:
        j=j.split()[0]+' '+j.split()[1]+' '+re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
    i['date']=date
    i['year']=int(str(i.get('date'))[:4])

In [316]:
len(mutation_list_raw)

34

In [317]:
mutation_list_raw[0]

{'AB': 'The clusterin (CLU) rs11136000 CC genotype is a probable risk factor for Alzheimer\'s disease (AD). CLU, also known as the apolipoprotein J gene, shares certain properties with the apolipoprotein E (APOE) gene with a well-established relationship with AD. This study aimed to determine whether the electrophysiological patterns of brain activation during the letter fluency task (LFT) depend on CLU genotypes in adults without dementia. Previous studies have shown that LFT performance involves activation of the frontal cortex. We examined EEG alpha1 and alpha2 band desynchronization in the frontal regions during the LFT in 94 nondemented individuals stratified by CLU (rs11136000) genotype. Starting at 30 years of age, CLU CC carriers exhibited more pronounced task-related alpha2 desynchronization than CLU CT&TT carriers in the absence of any differences in LFT performance. In CLU CC carriers, alpha2 desynchronization was significantly correlated with age. Increased task-related act

In [318]:
Mutation_raw=copy.deepcopy(mutation_list_raw)

In [319]:
collections_Mutation_raw.insert_many(Mutation_raw)

# Mutation_all

In [320]:
Mutation_testcopy=list(collections_Mutation_raw.find())

In [321]:
len(Mutation_testcopy)

6915

In [322]:
for i in Mutation_testcopy:
    i['type']='Mutation'
    i["PMID"]=[i.get("PMID")]
    i["sentence"]=[[i.get("sentence")]]
    pmid=i.get('PMID')[0]
    i['DP']=[i.get("DP")]
    i['AB']=[i.get("AB")]
    i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+pmid
    i['TI']=[i.get("TI")]
    i['target']=[[i.get('word')]]
    i["date"]=[i.get("date")]
    i["year"]=[i.get("year")]
    i['JT']=[i.get('JT')]

In [323]:
Mutation_mid={}
for i in Mutation_testcopy:
    if not Mutation_mid.get(i.get("entity")):
        Mutation_mid[i.get("entity")]=i
    else:
        if i.get('PMID')[0] in Mutation_mid.get(i.get('entity')).get('PMID'):

            ind=Mutation_mid.get(i.get('entity')).get('PMID').index(i.get('PMID')[0])
            if i.get("sentence")[0][0] in Mutation_mid.get(i.get('entity')).get('sentence')[ind]:
                continue
            else:
                Mutation_mid[i.get('entity')]['sentence'][ind] = Mutation_mid.get(i.get('entity')).get('sentence')[ind] + i.get("sentence")[0]
                Mutation_mid[i.get('entity')]['target'][ind] = Mutation_mid.get(i.get('entity')).get('target')[ind] + i.get("target")[0]
        else:
            Mutation_mid[i.get("entity")]["PMID"]=Mutation_mid.get(i.get("entity")).get("PMID")+i.get("PMID")
            Mutation_mid[i.get("entity")]["sentence"]=Mutation_mid.get(i.get("entity")).get("sentence")+i.get("sentence")
            Mutation_mid[i.get("entity")]["AB"]=Mutation_mid.get(i.get("entity")).get("AB")+i.get("AB")
            Mutation_mid[i.get("entity")]["DP"]=Mutation_mid.get(i.get("entity")).get("DP")+i.get("DP")
            Mutation_mid[i.get("entity")]["TI"]=Mutation_mid.get(i.get("entity")).get("TI")+i.get("TI")
            Mutation_mid[i.get("entity")]["target"]=Mutation_mid.get(i.get("entity")).get("target")+i.get("target")
            Mutation_mid[i.get("entity")]["date"]=Mutation_mid.get(i.get("entity")).get("date")+i.get("date")
            Mutation_mid[i.get("entity")]["year"]=Mutation_mid.get(i.get("entity")).get("year")+i.get("year")
            Mutation_mid[i.get("entity")]["JT"]=Mutation_mid.get(i.get("entity")).get("JT")+i.get("JT")

In [324]:
Mutation_all=list(Mutation_mid.values())

In [325]:
for i in Mutation_all:
    i['allyears']=[]
    i['datestring']=[]
    for j in i.get('date'):
        if int(str(j)[:4]) not in i.get('allyears'):
            i['allyears']= i.get('allyears')+[int(str(j)[:4])]
    for j in i.get('date'):
        if str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:] not in i.get('datestring'):
            i['datestring']=i.get('datestring')+[str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:]]
    i['allyears']=sorted(i.get('allyears'), reverse=True)
    i['datestring']=sorted(i.get('datestring'), reverse=True)
    upperlist=set()
    for j in sum(i.get('target'),[]):
        upperlist.add(j.upper())
    i['targetshapes']=list(upperlist)
    i['addtime']=now_time

In [326]:
len(Mutation_all)

2680

# Disease_raw

In [327]:
collections_Disease_raw=db.Disease_raw

In [328]:
Disease_pubtator_addpmid={}
for i in pubtator_raw:
    if i.get('type')=='Disease':
        if Disease_pubtator_addpmid.get(i.get('id')):
            Disease_pubtator_addpmid[i.get('id')]=Disease_pubtator_addpmid.get(i.get('id'))+[i]
        else:
            Disease_pubtator_addpmid[i.get('id')]=[i]

In [329]:
Disease_output_json=[]
for i,j in Disease_pubtator_addpmid.items():
    Disease_output_json+=j

In [330]:
len(Disease_output_json)

9539

In [331]:
Disease_mesh=pickle.load(open('../disease/Disease.pkl','rb'))

In [332]:
Disease_mesh=Disease_mesh[1:]

In [333]:
for i in Disease_mesh:
    if i.get('text')=='Uveitis, Suppurative':
        i['MeSHUniqueID']='D015829'

In [334]:
for i in Disease_mesh:
    if i.get('text')=='Hearing Loss':
        print(i)

{'text': 'Hearing Loss', 'url': '/mesh/68034381', 'MeSHUniqueID': 'D034381', 'type': 'Disease'}
{'text': 'Hearing Loss', 'url': '/mesh/68034381', 'MeSHUniqueID': 'D034381', 'type': 'Disease'}
{'text': 'Hearing Loss', 'url': '/mesh/68034381', 'MeSHUniqueID': 'D034381', 'type': 'Disease'}


In [335]:
diseaselist=[]
notdiseaselist=[]
for i in Disease_mesh:
    if i.get('MeSHUniqueID'):
        diseaselist.append({'MESH':'MESH:'+i.get('MeSHUniqueID'),'type':i.get('type'),'url':i.get('url'),'text':i.get('text')})
    else:
        notdiseaselist.append(i)

In [336]:
len(diseaselist)

12791

In [337]:
len(notdiseaselist)

0

In [338]:
disease2classdict={}
count=0
for i in diseaselist:
    if not disease2classdict.get(i.get('MESH')):
        disease2classdict[i.get('MESH')]=[i.get('type')]
    else:
        if i.get('type') in disease2classdict.get(i.get('MESH')):
            pass
        else:
            disease2classdict[i.get('MESH')]=disease2classdict.get(i.get('MESH'))+[i.get('type')]
            count+=1
            if count<=5:
                print(i.get('MESH')+'\t'+str(disease2classdict.get(i.get('MESH'))))

In [339]:
disease2subclassdict={}
count=0
for i in diseaselist:
    disease2subclassdict[i.get('MESH')]=i.get('text')

In [340]:
disease2urldict={}
count=0
for i in diseaselist:
    disease2urldict[i.get('MESH')]=i.get('url')

In [341]:
len(disease2classdict)

4915

In [342]:
for i in Disease_output_json:
    i['PMID']=i.get('id')
    i['word']=i.get('text')
    del i['id']
    del i['text']

In [343]:
Disease_final=[]
Disease_dropout=[]

In [344]:
for i in Disease_output_json:
    if i.get('identifier') in disease2classdict.keys():
        Disease_final.append(dict(i,**{'class':disease2classdict.get(i.get('identifier')),'subclass':disease2subclassdict.get(i.get('identifier')),'url':disease2urldict.get(i.get('identifier'))}))
    else:
        Disease_dropout.append(i)

In [345]:
len(Disease_output_json)

9539

In [346]:
len(Disease_dropout)

1785

In [347]:
Disease_deduplicated=set()
Disease_deduplicated_count=0
disease_list_raw=[]
for i in Disease_final:
    k=i.get('PMID')
    i['DP']=rawpickwithAB.get(k).get('DP')
    i['AB']=rawpickwithAB.get(k).get('AB')
    i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+k
    i['TI']=rawpickwithAB.get(k).get('TI')
    i['entity']=i.get('subclass')
    i['target']=i.get('word')
    i['type']='Disease'
    i['JT']=rawpickwithAB.get(k).get('JT')
    if i.get('identifier') and re.match(r'MESH:',i.get('identifier')):
        i['MESH']=i.get('identifier')[5:]
    else:
        i['MESH']=''
    # tuples=(i.get("PMID"),i.get("entity"),i.get("word"),i.get("sentence"))
    tuples=(i.get("PMID"),i.get("entity"),i.get("word"),i.get("sentence"))
    if tuples not in Disease_deduplicated:
        Disease_deduplicated.add(tuples)
        disease_list_raw.append(i)
    else:
        Disease_deduplicated_count+=1
        continue

In [348]:
Disease_deduplicated_count

580

In [349]:
disease2ncbi={}
for i in disease_list_raw:
    disease2ncbi[i.get('url')]=''

In [350]:
len(disease2ncbi)

677

In [351]:
disease_converter=pickle.load(open('../ncbi/Disease_converter.pkl','rb'))

In [352]:
len(disease_converter)

3347

In [353]:
def get_disease_info(i,final_diseases_inte):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"
    }
    url = 'https://www.ncbi.nlm.nih.gov' + i
    res=requests.get(url=url,headers=headers)
    html = etree.HTML(res.content)
    ncbisummary={
    # 'numbersOfLiterature' : len(i.get('PMID')),
    'Summary' : ''.join(html.xpath('//*[@class="mesh_ds_scope_note"]/text()'))
    }
    final_diseases_inte.update({i:ncbisummary})
    time.sleep(0.5)

In [354]:
judge=False
p = Pool(10)
manager=Manager()
final_diseases_inte = manager.dict()
for i in disease2ncbi.keys():
    if not disease_converter.get(i):
        p.apply_async(get_disease_info, args=(i,final_diseases_inte))
        judge=True
p.close()
p.join()
if judge:
    disease_converter.update(dict(final_diseases_inte))
    pickle.dump(disease_converter, open('../ncbi/Disease_converter.pkl','wb'))

In [355]:
len(disease_converter)

3352

In [356]:
for i in range(len(disease_list_raw)):
    disease_list_raw[i]=dict(disease_list_raw[i],**disease_converter.get(disease_list_raw[i].get('url')))

In [357]:
Disease_deduplicated_dropout=set()
Disease_deduplicated_dropout_count=0
Disease_dropout_raw=[]
for i in Disease_dropout:
    k=i.get('PMID')
    i['DP']=rawpickwithAB.get(k).get('DP')
    i['AB']=rawpickwithAB.get(k).get('AB')
    i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+k
    i['TI']=rawpickwithAB.get(k).get('TI')
    i['entity']=i.get('word')
    i['target']=i.get('word')
    i['type']='Disease'
    i['JT']=rawpickwithAB.get(k).get('JT')
    if i.get('identifier') and re.match('MESH:',i.get('identifier')):
        i['MESH']=i.get('identifier')[5:]
    else:
        i['MESH']=''
    # tuples=(i.get("PMID"),i.get("entity"),i.get("word"),i.get("sentence"))
    tuples=(i.get("PMID"),i.get("entity"),i.get("word"),i.get("sentence"))
    if tuples not in Disease_deduplicated_dropout:
        Disease_deduplicated_dropout.add(tuples)
        Disease_dropout_raw.append(i)
    else:
        Disease_deduplicated_dropout_count+=1
        continue

In [358]:
for i in Disease_dropout_raw:
    i['Summary']=''

In [359]:
len(Disease_dropout_raw)

1697

In [360]:
Disease_raw=disease_list_raw+Disease_dropout_raw

In [361]:
len(Disease_raw)

8871

In [362]:
dropout=[]
months=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
for i in Disease_raw:
    j=i.get('DP')
    if re.search('[S,s]upplement',j,flags=re.M|re.I):
        j=re.sub('[S,s]upplement.*','',j,flags=re.M|re.I)
    j=re.sub('\s?-\s?','-',j)
    j=re.sub('\.','',j)
    j=re.sub('Aout','Aug',j)
    if len(j.split(' '))==3:
        try:
            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
        except:
            if j.split()[1] in months:
                if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                    j=j.split()[0]+' '+j.split()[1]+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                        print('***************************************ERROR**********************************************')
                else:
                    j=j.split()[0]+' '+j.split()[1]+' '+ '1'
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        print(j)
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
            elif j.split()[1] not in months:
                if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                        middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                    else:
                        middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if middle in months:
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+middle+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+middle+' '+ '1'
                    elif re.match(r'Sum(mer)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Jun'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Jun'+' '+ '1'
                    elif re.match(r'Spr(ing)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Mar'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Mar'+' '+ '1'
                    elif re.match(r'Win(ter)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Dec'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Dec'+' '+ '1'    
                    elif re.match(r'Aut(umn)?',middle) or \
                    re.match(r'Fall',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Sep'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Sep'+' '+ '1'      
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        print(j)
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                else:
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    elif len(j.split())==2:
        if re.match(r'Sum(mer)?',j.split()[1]):
            date=int(j[:4]+'0601')
        elif re.match(r'Spr(ing)?',j.split()[1]):
            date=int(j[:4]+'0301')
        elif re.match(r'Win(ter)?',j.split()[1]):
            date=int(j[:4]+'1201')
        elif re.match(r'Aut(umn)?',j.split()[1]) or re.match(r'Fall',j.split()[1]):
            date=int(j[:4]+'0901')
        elif re.match(r'Sum(mer)?',j.split()[0]):
            date=int(j[-4:]+'0601')
        elif re.match(r'Spr(ing)?',j.split()[0]):
            date=int(j[-4:]+'0301')
        elif re.match(r'Win(ter)?',j.split()[0]):
            date=int(j[-4:]+'1201')
        elif re.match(r'Aut(umn)?',j.split()[0]) or re.match(r'Fall',j.split()[0]):
            date=int(j[-4:]+'0901')
        else:
            try:
                date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b'), '%Y%m'+'01'))
            except:
                if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                        middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                    else:
                        middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if middle in months:
                        j=j.split()[0]+' '+middle+' '+ '1'
                    elif re.match(r'Sum(mer)?',middle):
                        j=j.split()[0]+' '+'Jun'+' '+ '1'
                    elif re.match(r'Spr(ing)?',middle):
                        j=j.split()[0]+' '+'Mar'+' '+ '1'
                    elif re.match(r'Win(ter)?',middle):
                        j=j.split()[0]+' '+'Dec'+' '+ '1'    
                    elif re.match(r'Aut(umn)?',middle) or \
                    re.match(r'Fall',middle):
                        j=j.split()[0]+' '+'Sep'+' '+ '1'      
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                elif re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    midd=re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if len(midd)!=2:
                        midd='0'+midd
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+midd+'01'))
                else:
                    print(j)
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    elif len(j.split())==1:
        j=j[:4]
        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    else:
        j=j.split()[0]+' '+j.split()[1]+' '+re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
    i['date']=date
    i['year']=int(str(i.get('date'))[:4])

In [363]:
collections_Disease_raw.insert_many(Disease_raw)

# Disease_all

In [364]:
Disease_raw_list=list(collections_Disease_raw.find())

In [365]:
for i in Disease_raw_list:
    if i.get('word').upper()=="ALZHEI-MER'S DISEASE":
        i['entity']='Alzheimer Disease'

In [366]:
for i in Disease_raw_list:
    i["PMID"]=[i.get("PMID")]
    i["sentence"]=[[i.get("sentence")]]
    pmid=i.get('PMID')[0]
    i['DP']=[i.get("DP")]
    i['AB']=[i.get("AB")]
    i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+pmid
    i['TI']=[i.get("TI")]
    i['target']=[[i.get('word')]]
    i["date"]=[i.get("date")]
    i["year"]=[i.get("year")]
    i['JT']=[i.get('JT')]

In [367]:
i

{'_id': ObjectId('6459f29748e9c8c06012c085'),
 'AB': ['OBJECTIVES: Exposures to adverse events are associated with impaired later-life psychological health. While these associations depend on the type of event, the manner in which associations for different event types depend on when they occur within the life course has received less attention. We investigated associations between counts of adverse events over the life course, and wellbeing and mental health outcomes in older people, according to their timing (age of occurrence), orientation (self or other) and, both their timing and orientation. DESIGN: Linear and logistic random-effects models for repeated observations. SETTING: England, 2002-2015. PARTICIPANTS: A total of 4,208 respondents aged >50 years with 22,146 observations across Waves 1-7 of the English Longitudinal Study of Ageing. MEASUREMENTS: Cumulative adversity was measured by counts of 16 types of events occurring within four age ranges over the life course using retr

In [368]:
Disease_mid={}
for i in Disease_raw_list:
    if not Disease_mid.get(i.get("entity")):
        Disease_mid[i.get("entity")]=i
    else:
        if i.get('PMID')[0] in Disease_mid.get(i.get('entity')).get('PMID'):
            ind=Disease_mid.get(i.get('entity')).get('PMID').index(i.get('PMID')[0])
            if i.get("sentence")[0][0] in Disease_mid.get(i.get('entity')).get('sentence')[ind]:
                continue
            else:
                Disease_mid[i.get('entity')]['sentence'][ind] = Disease_mid.get(i.get('entity')).get('sentence')[ind] + i.get("sentence")[0]
                Disease_mid[i.get('entity')]['target'][ind] = Disease_mid.get(i.get('entity')).get('target')[ind] + i.get("target")[0]
        else:
            Disease_mid[i.get("entity")]["PMID"]=Disease_mid.get(i.get("entity")).get("PMID")+i.get("PMID")
            Disease_mid[i.get("entity")]["sentence"]=Disease_mid.get(i.get("entity")).get("sentence")+i.get("sentence")
            Disease_mid[i.get("entity")]["AB"]=Disease_mid.get(i.get("entity")).get("AB")+i.get("AB")
            Disease_mid[i.get("entity")]["DP"]=Disease_mid.get(i.get("entity")).get("DP")+i.get("DP")
            Disease_mid[i.get("entity")]["TI"]=Disease_mid.get(i.get("entity")).get("TI")+i.get("TI")
            Disease_mid[i.get("entity")]["target"]=Disease_mid.get(i.get("entity")).get("target")+i.get("target")
            Disease_mid[i.get("entity")]["date"]=Disease_mid.get(i.get("entity")).get("date")+i.get("date")
            Disease_mid[i.get("entity")]["year"]=Disease_mid.get(i.get("entity")).get("year")+i.get("year")
            Disease_mid[i.get("entity")]["JT"]=Disease_mid.get(i.get("entity")).get("JT")+i.get("JT")

In [369]:
Disease_output_json_final=list(Disease_mid.values())

In [370]:
len(Disease_output_json_final)

17750

In [371]:
for i in Disease_output_json_final:
    i['allyears']=[]
    i['datestring']=[]
    for j in i.get('date'):
        if int(str(j)[:4]) not in i.get('allyears'):
            i['allyears']= i.get('allyears')+[int(str(j)[:4])]
    for j in i.get('date'):
        if str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:] not in i.get('datestring'):
            i['datestring']=i.get('datestring')+[str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:]]
    i['allyears']=sorted(i.get('allyears'), reverse=True)
    i['datestring']=sorted(i.get('datestring'), reverse=True)
    upperlist=set()
    for j in sum(i.get('target'),[]):
        upperlist.add(j.upper())
    i['targetshapes']=list(upperlist)
    i['addtime']=now_time

In [372]:
Disease_all=copy.deepcopy(Disease_output_json_final)

In [373]:
len(Disease_all)

17750

In [376]:
names=locals()

In [377]:
entity_pairs=['Gene','RNA','Carbohydrate','Lipid','Peptide', 'Pharmaceutical_Preparations', 'Protein', 'Toxin','Mutation', 'Disease']

In [378]:
for i in entity_pairs:
    pickle.dump(names[i+'_all'],open('results/step2/'+i+'_all.pkl','wb'))

In [380]:
for i in entity_pairs:
    names[i+'_all']=pickle.load(open('results/step2/'+i+'_all.pkl','rb'))

In [385]:
Gene_all[10].keys()

dict_keys(['_id', 'AB', 'type', 'identifier', 'sentence', 'sentenceSize', 'sentenceIndex', 'entity', 'ner', 'PMID', 'word', 'jnlpba_tags', 'bc5cdr_tags', 'bionlp13cg_tags', 'ScispaCy', 'bern', 'DP', 'URL', 'TI', 'target', 'JT', 'Official_Symbol', 'Official_Full_Name', 'HGNC_ID', 'HGNC_URL', 'Ensembl_ID', 'Ensembl_URL', 'MIM_ID', 'MIM_URL', 'AllianceGenome_ID', 'AllianceGenome_URL', 'Gene_type', 'Lineage', 'Also_known_as', 'Summary', 'NCBI_ID', 'NCBI_URL', 'date', 'year', 'addtime', 'allyears', 'datestring', 'targetshapes'])

In [386]:
RNA_all[10].keys()

dict_keys(['_id', 'AB', 'type', 'identifier', 'sentence', 'sentenceSize', 'sentenceIndex', 'entity', 'ner', 'specific_type', 'PMID', 'word', 'DP', 'URL', 'TI', 'target', 'Official_Symbol', 'Official_Full_Name', 'HGNC_ID', 'HGNC_URL', 'Ensembl_ID', 'Ensembl_URL', 'MIM_ID', 'MIM_URL', 'AllianceGenome_ID', 'AllianceGenome_URL', 'Gene_type', 'Lineage', 'Also_known_as', 'Summary', 'Expression', 'NCBI_ID', 'NCBI_URL', 'date', 'year', 'JT', 'addtime', 'allyears', 'datestring', 'targetshapes'])

In [387]:
all_es=[]
mu=0
di=0
noturl=[]
for i in Gene_all:
    if i.get('Summary'):
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'Also_known_as':i.get('Also_known_as'),'Summary':sent_tokenize(i.get('Summary'))[0],'NCBI_URL':i.get('NCBI_URL'),'NCBI_ID':i.get('NCBI_ID'),'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
    else:
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'Also_known_as':i.get('Also_known_as'),'Summary':'','NCBI_URL':i.get('NCBI_URL'),'NCBI_ID':i.get('NCBI_ID'),'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
for i in RNA_all:
    if i.get('Summary'):
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'specific_type':i.get('specific_type'),'Also_known_as':i.get('Also_known_as'),'Summary':sent_tokenize(i.get('Summary'))[0],'NCBI_URL':i.get('NCBI_URL'),'NCBI_ID':i.get('NCBI_ID'),'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
    else:
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'specific_type':i.get('specific_type'),'Also_known_as':i.get('Also_known_as'),'Summary':'','NCBI_URL':i.get('NCBI_URL'),'NCBI_ID':i.get('NCBI_ID'),'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
for i in Carbohydrate_all:
    if i.get('Summary'):
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'MeshID':i.get('MESH'),'Summary':sent_tokenize(i.get('Summary'))[0],'url':'https://www.ncbi.nlm.nih.gov'+i.get('url'),'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
    else:
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'MeshID':i.get('MESH'),'Summary':'','url':'https://www.ncbi.nlm.nih.gov'+i.get('url'),'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
for i in Lipid_all:
    if i.get('Summary'):
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'MeshID':i.get('MESH'),'Summary':sent_tokenize(i.get('Summary'))[0],'url':'https://www.ncbi.nlm.nih.gov'+i.get('url'),'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
    else:
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'MeshID':i.get('MESH'),'Summary':'','url':'https://www.ncbi.nlm.nih.gov'+i.get('url'),'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
for i in Peptide_all:
    if i.get('Summary'):
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'MeshID':i.get('MESH'),'Summary':sent_tokenize(i.get('Summary'))[0],'url':'https://www.ncbi.nlm.nih.gov'+i.get('url'),'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
    else:
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'MeshID':i.get('MESH'),'Summary':'','url':'https://www.ncbi.nlm.nih.gov'+i.get('url'),'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
for i in Pharmaceutical_Preparations_all:
    if i.get('Summary'):
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'MeshID':i.get('MESH'),'Summary':sent_tokenize(i.get('Summary'))[0],'url':'https://www.ncbi.nlm.nih.gov'+i.get('url'),'_id':'PharmaceuticalPreparations'+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
    else:
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'MeshID':i.get('MESH'),'Summary':'','url':'https://www.ncbi.nlm.nih.gov'+i.get('url'),'_id':'PharmaceuticalPreparations'+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
for i in Protein_all:
    if i.get('Summary'):
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'MeshID':i.get('MESH'),'Summary':sent_tokenize(i.get('Summary'))[0],'url':'https://www.ncbi.nlm.nih.gov'+i.get('url'),'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
    else:
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'MeshID':i.get('MESH'),'Summary':'','url':'https://www.ncbi.nlm.nih.gov'+i.get('url'),'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
for i in Toxin_all:
    if i.get('Summary'):
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'MeshID':i.get('MESH'),'Summary':sent_tokenize(i.get('Summary'))[0],'url':'https://www.ncbi.nlm.nih.gov'+i.get('url'),'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
    else:
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'MeshID':i.get('MESH'),'Summary':'','url':'https://www.ncbi.nlm.nih.gov'+i.get('url'),'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
for i in Mutation_all:
    if i.get('Genomic View'):
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'Genomic_View':i.get('Genomic View'),'Alleles':i.get('Alleles'),'Position':i.get('Position'),'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
    else:
        mu+=1
for i in Disease_all:
    if i.get('url'):
        if i.get('Summary'):
            all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'MeshID':i.get('MESH'),'url':'https://www.ncbi.nlm.nih.gov'+i.get('url'),'Summary':sent_tokenize(i.get('Summary'))[0],'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
        else:
            all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'MeshID':i.get('MESH'),'url':'https://www.ncbi.nlm.nih.gov'+i.get('url'),'Summary':'','_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
    else:
        di+=1
        noturl.append(i)

In [388]:
all_es_copy=copy.deepcopy(all_es)

In [389]:
len(all_es_copy)

11127

In [390]:
for i in all_es:
    i['allwordsdict']={}
    if i.get('Also_known_as'):
        i['allwords']=i.get('allwords',[])+i.get('Also_known_as').split('; ')
        for l in i.get('Also_known_as').split('; '):
            i['allwordsdict'][l.upper()]=l
    for j in i.get('targetshapes'):
        if j.upper() != i.get('entity').upper() and not i.get('allwordsdict').get(j.upper()):
            i['allwords']=i.get('allwords',[])+[j.upper()]
            i['allwordsdict'][j.upper()]=j
    if i.get('targetshapes'):
        del i['targetshapes']
    del i['allwordsdict']

In [391]:
all_es[-1000]

{'entity': 'Periprosthetic Fractures',
 'PMID': ['30539204'],
 'literature': 1,
 'allyears': [2019],
 'type': 'Disease',
 'MeshID': 'D057068',
 'url': 'https://www.ncbi.nlm.nih.gov/mesh/68057068',
 'Summary': 'Fractures around joint replacement prosthetics or implants.',
 '_id': 'Disease_Periprosthetic Fractures',
 'datest': 20190301,
 'allwords': ['PERIPROSTHETIC JOINT INFECTIONS', 'PJI']}

In [392]:
len(all_es)

11127

In [393]:
noturls=[]
for i in Disease_all:
    if not i.get('url'):
        noturls.append(i)

In [394]:
len(noturls)

14398

In [395]:
len(Disease_all)

17750

In [396]:
json.dump(all_es,open('results/step2/all_es_lack.json','w'))

In [397]:
entity_pairs=['Gene','RNA','Carbohydrate','Lipid','Peptide', 'Pharmaceutical_Preparations', 'Protein', 'Toxin','Mutation', 'Disease']

In [398]:
all_es_result=[]
for j in entity_pairs:
    for i in names[j+'_all']:
        if i.get('specific_type'):
            all_es_result.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'target':i.get('target'),'date':i.get('date'),'type':i.get('specific_type'),'sentence':i.get('sentence'),'_id':i.get('type')+'_'+i.get('entity')})
        else:
            if i.get('type')=='Disease':
                if i.get('url'):
                    all_es_result.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'target':i.get('target'),'date':i.get('date'),'type':i.get('type'),'sentence':i.get('sentence'),'_id':i.get('type')+'_'+i.get('entity')})
            elif i.get('type')=='Mutation':
                if i.get('Genomic View'):
                    all_es_result.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'target':i.get('target'),'date':i.get('date'),'type':i.get('type'),'sentence':i.get('sentence'),'_id':i.get('type')+'_'+i.get('entity')})
            else:
                all_es_result.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'target':i.get('target'),'date':i.get('date'),'type':i.get('type'),'sentence':i.get('sentence'),'_id':i.get('type')+'_'+i.get('entity')})

In [399]:
jcr=pd.read_csv("../JCR/jcr2022.tsv",sep="\t")

In [400]:
collections_pubtator_info = db.pubtator_info

In [401]:
pubtator_info=list(collections_pubtator_info.find())

In [402]:
pubtator_info_dict={}
for i in pubtator_info:
    pubtator_info_dict[i.get('PMID')]=i

In [403]:
jt2if_all={}
for i in range(len(jcr)):
    if jt2if_all.get(jcr.iloc[i,1].lower()):
        if jt2if_all.get(jcr.iloc[i,1].lower())==0.0:
            jt2if_all[jcr.iloc[i,1].lower()]=jcr.iloc[i,7]
    else:
        jt2if_all[jcr.iloc[i,1].lower()]=jcr.iloc[i,7]
jt2if_all['aging']=jt2if_all.get('aging-us')

In [404]:
jt2if_brief={}
for i in range(len(jcr)):
    if jt2if_brief.get(jcr.iloc[i,2].lower()):
        if jt2if_brief.get(jcr.iloc[i,2].lower())==0.0:
            jt2if_brief[jcr.iloc[i,2].lower()]=jcr.iloc[i,7]
    else:
        jt2if_brief[jcr.iloc[i,2].lower()]=jcr.iloc[i,7]
jt2if_brief['aging']=jt2if_brief.get('aging-us')

In [405]:
jt2if5_all={}
for i in range(len(jcr)):
    if jt2if5_all.get(jcr.iloc[i,1].lower()):
        if jt2if5_all.get(jcr.iloc[i,1].lower())==0.0:
            jt2if5_all[jcr.iloc[i,1].lower()]=jcr.iloc[i,7]
    else:
        jt2if5_all[jcr.iloc[i,1].lower()]=jcr.iloc[i,9]
jt2if5_all['aging']=jt2if5_all.get('aging-us')

In [406]:
jt2if5_brief={}
for i in range(len(jcr)):
    
    if jt2if5_brief.get(jcr.iloc[i,2].lower()):
        if jt2if5_brief.get(jcr.iloc[i,2].lower())==0.0:
            jt2if5_brief[jcr.iloc[i,2].lower()]=jcr.iloc[i,7]
    else:
        jt2if5_brief[jcr.iloc[i,2].lower()]=jcr.iloc[i,9]
jt2if5_brief['aging']=jt2if5_brief.get('aging-us')

In [407]:
for i in all_es_result:
    iflist=[]
    if5list=[]
    TAlist=[]
    JTlist=[]
    yearlist=[]
    for j in i.get('PMID'):
        if jt2if_brief.get(pubtator_info_dict.get(j).get('TA').lower()):
            iflist.append(jt2if_brief.get(pubtator_info_dict.get(j).get('TA').lower()))
        elif jt2if_all.get(pubtator_info_dict.get(j).get('JT').lower()):
            iflist.append(jt2if_all.get(pubtator_info_dict.get(j).get('JT').lower()))
        else:
            iflist.append(0.0)
        if jt2if5_brief.get(pubtator_info_dict.get(j).get('TA').lower()):
            if5list.append(jt2if5_brief.get(pubtator_info_dict.get(j).get('TA').lower()))
        elif jt2if5_all.get(pubtator_info_dict.get(j).get('JT').lower()):
            if5list.append(jt2if5_all.get(pubtator_info_dict.get(j).get('JT').lower()))
        else:
            if5list.append(0.0)
        TAlist.append(pubtator_info_dict.get(j).get('TA'))
        JTlist.append(pubtator_info_dict.get(j).get('JT'))
    for l in i.get('date'):
        yearlist.append(int(str(l)[:4]))
    i['IF']=iflist
    i['IF5']=if5list
    i['TA']=TAlist
    i['JT']=JTlist
    i['year']=yearlist

In [408]:
json.dump(all_es_result,open('results/step2/all_es_result.json','w'))

In [409]:
len(all_es_result)

11127

In [410]:
blank=[{'Module':'','Description':'','Entity_Relation':'','Username':'','Email':''}]

In [411]:
json.dump(blank,open('results/step2/feedback.json','w'))